In [ ]:
import pandas as pd
import re
from datetime import date 
from bs4 import BeautifulSoup
import requests
import time 
from requests_html import HTMLSession
import json
import random
import numpy as np
from sqlalchemy import create_engine, text
import math

In [ ]:
# Read data for maching HP model and HP Class
# path = r"C:\Users\TaYu430\anaconda3\envs\webscrap\My Scripts\Star rating scrape URL and info - NPI.xlsx"
# path = r"C:\Users\TaYu430\anaconda3\envs\webscrap\My Scripts\Star rating scrape URL and info - NPI.xlsx"
path = r"Star rating scrape URL and info - NPI.xlsx"
sheets = 'data_new'
df_amazon = pd.read_excel(path, sheet_name = sheets)
df_amazon['Comp Model number'] =  df_amazon['Comp Model number'].astype(str)
df_amazon['HP Model Number'] =  df_amazon['HP Model Number'].astype(str)
# df_amazon
df_amazon.head() 

In [ ]:
# Dataframe to store all scraping results
# path = rpath = r"C:\Users\TaYu430\anaconda3\envs\webscrap\My Scripts\Star rating scrape URL and info - NPI.xlsx"
# path = rpath = r"C:\Users\TaYu430\anaconda3\envs\webscrap\My Scripts\Star rating scrape URL and info - NPI.xlsx"
path = rpath = r"Star rating scrape URL and info - NPI.xlsx"
sheets = 'review_template'
review_template = pd.read_excel(path, sheet_name = sheets)
review_template

# Amazon

## Function (New Soup with Cookies--ANAM)

In [ ]:
#The list extracted_reviews has all the dictionaries of review

def get_soup(url):
    headers = {
        "Host": "www.amazon.com",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1"
    }
    
    cookies = {
        "id_pkel": "n0",
        "session-token": "\"JCjKSSKTw7jVnrHceNZ1Fu9RrQvAIa+s2p8SaNUNOmiDV++3Zu+IzXgcDxVF1R7afT+dELYyTqw1tjXikJRvLOqF/fB796ptA6w2imoOQW3RXiNVLk1+/wQ3Gyv7CO9mTHo6K5nznt80rWbD8JPTrkrA4cfYIM/7bOVpd5LrmWEetgku5048aKmWOHN1mLV81SEzBdOeRFF+bn2FFhiDhuX694izkkgPtGfiJl8ZhsVl/bDHuPhwXZW3mGLo0NHuCRCiV3y04371l+4ezeInm8odlirZdgtdCeUgQX3BaHgUu0mdD/NTYNReC11kVYyjQy2PxiQ+H5J3yJ1mJKA2xP0si7I3yClRfE7Yox2LDLkmITohV7/0SQ==\"",
        "x-main": "\"DqP4xE7LeosRnDhmsoZxlQWlOSq0S@H3kfs6w5KT@IJvSqyW@FP5YDzCVNF2?Or@\"",
        "at-main": "Atza|IwEBILqIzWhp33lFOzPr-LefQfnaGe7DjvswGMC91WCpK7fv7j_HKjl4Nzwb3NssJedEoFPlU0o3FpNwpz28DesoYtVg7TZVLJt0vKqqOyZf0QAqx-IdUEezH4MAk_hI0Mvc2NUQavOYlPjRHjsD4u0ScUEL5eyOz9Phfczu3nQbebANxVYLZLmJh5vG_Q6QBV6q_dA6MoN8ZO57k-K4N3KVn0RvL6JMfsZT6nlChD-aTpxDBg",
        "sess-at-main": "\"gb/7B4csGrwlKApsj93tsowvv4Be5sxIMfJBWCOAl0s=\"",
        "sst-main": "Sst1|PQEUieAFX8NNHVzhl1NKjGddCbufYeBVV0Uhty2ygjXgz6IpJdCudqKsA5hI3uA9mtpAJ8CYGj0G7xLU7zSG8uzF9LPjKaRrEqJ4DKYJunvj5gG_35tyPPC5gsUElqjjrq82W078uJs491QbzjBJ_HVygvByC60QKjF5NU1gMf43JYgnJm2e1I6S01sLrkP0MlJV4aRznUDYJ7J0ArUh7ytFY1TlUrlLk4NhfgmX6DrthT7orjIxRvc1LD-TXFRkHKZOGSdP7D-7Ub2qULE_YibbRbuvjj0YOe8Uh_oFsmbtvp0",
        "i18n-prefs": "USD",
        "lc-main": "en_US",
        "sp-cdn": "\"L5Z9:IN\"",
        "ubid-main": "132-2235232-6889017",
        "session-id-time": "2082787201l",
        "session-id": "131-3748265-2522507",
        "csm-hit": "tb:ZK52AJH8KQJXFXD7BSAA+s-7N9Q8F0T2KMBKYCFNT10|1724917542317&t:1724917542317&adb:adblk_no"
    }

    req = requests.get(url, headers=headers, cookies=cookies)
    soup = BeautifulSoup(req.content, "html.parser")
    return soup
 
  
def amazon_review(soup, url):    
    review = {} #create a dictionary
    extracted_reviews = []  #create an list
    try:
        model = soup.title.text.replace("Amazon.com: Customer reviews: ","")    
    except AttributeError: 
        try:
            model = soup.find("a", attrs={"data-hook": "product-link"}).string.strip()  
        except AttributeError: 
            model = soup.find("div", attrs={"class": "a-row product-title"}).string.strip()  
  
    reviews = soup.find_all("div", {"data-hook": "review"})
    
    
    for item in reviews:    
        review = {    
            'Model': model,    
            'Review date': item.find('span', {'data-hook': 'review-date'}).text.replace('Reviewed in', '').split('on')[1],     
            "Review Content": item.find("span", {'data-hook': "review-body"}).text.strip(),  
            "URL" : url  
        }
        
        
  
        try:    
            review["Review rating"] = float(item.find("i", {"data-hook": "review-star-rating"}).text.replace("out of 5 stars", "").strip())    
        except AttributeError:    
            review["Review rating"] = float(item.find("span", {"class": "a-icon-alt"}).text.replace("out of 5 stars", "").strip())    
  
        try:    
            review['Review title']  = item.find("a", {'data-hook': "review-title"}).text.strip()    
        except AttributeError:    
            review['Review title']  = item.find("span", {'data-hook': "review-title"}).text.strip()    
  
        try:    
            review["Verified Purchase or not"] = item.find("span", {'data-hook': "avp-badge"}).text.strip()    
        except AttributeError:    
            review["Verified Purchase or not"] = None    
  
        try:      
            review["Review name"] = item.find("span", {'class': "a-profile-name"}).string.strip()  
        except AttributeError:        
            review["Review name"] = None  
  
        try:    
            review["People_find_helpful"] = item.find("span", {'data-hook': "helpful-vote-statement"}).text.strip()    
        except AttributeError:    
            review["People_find_helpful"] = None  
            
        try:
            seeding= item.find("span", {'class': "a-size-mini a-color-link c7yBadgeAUI c7yTopDownDashedStrike c7y-badge-text a-text-normal c7y-badge-link c7y-badge-vine-voice a-text-bold"}).text.strip() 
            if seeding:
               review['Seeding or not'] = seeding
            else:
                raise AttributeError
        except AttributeError:  
            try: 
                review['Seeding or not'] = item .find('span', {'class': 'a-color-success a-text-bold'}, text='Vine Customer Review of Free Product')

            except AttributeError:
                review['Seeding or not'] = None


        try:
            review['Aggregation_Flag'] = item.find("a", {"data-hook": "format-strip"}).text.strip()
        except AttributeError:   
             review['Aggregation_Flag'] = None

            
    
        #extract one page of review 
        extracted_reviews.append(review)    
    
  
    return extracted_reviews 


## HP Review

In [ ]:
## Run for all CPE, URL from excel
# path =  r"C:\Users\TaYu430\anaconda3\envs\webscrap\My Scripts\Star rating scrape URL and info - NPI.xlsx"
# sheets = 'Amazon'
# amazon_url = pd.read_excel(path, sheet_name = sheets)

# all_list = amazon_url['HP URL'].to_list()

# urls = []
# for value in all_list:
#     if value is not None and value not in urls:
#         urls.append(value)
# print(len(urls))

In [ ]:
## ad hoc run for CPE

urls = [
    ## Malbec
'https://www.amazon.com/product-reviews/B08WCDLKFK/ref=cm_cr_arp_d_viewopt_sr?formatType=current_format',
# 'https://www.amazon.com/product-reviews/B0B4PMYCWC/ref=cm_cr_arp_d_viewopt_sr?formatType=current_format',

# ## Manhantten
# 'https://www.amazon.com/product-reviews/B08QR6P8KV/ref=cm_cr_arp_d_viewopt_fmt?formatType=current_format',
# 'https://www.amazon.com/product-reviews/B08WC8VD8G/ref=cm_cr_arp_d_viewopt_fmt?formatType=current_format'

## CISS
    # 'https://www.amazon.com/product-reviews/B0BL466Y41/ref=cm_cr_arp_d_viewopt_sr?',
    # 'https://www.amazon.com/product-reviews/B09TPZ3HLZ/ref=cm_cr_arp_d_viewopt_srt?'

## Novellie
# 'https://www.amazon.com/product-reviews/B09NS5HDWY'


]

import datetime 
from datetime import datetime
star = ['one','two','three','four','five'] 
max_retry_attempts = 10
all_reviews = []

for link in urls:
    print(link)
    for y in star:
        found_reviews = True
        for page in range(1, 10):
            retry_attempts = 0
            while found_reviews is True:
                try:
                    url = f'{link}/ref=cm_cr_arp_d_viewopt_fmt?filterByStar={y}_star&pageNumber={page}&sortBy=recent&formatType=current_format'  
                    print(url)
                    print('Page:',page, f'{y} star')
                    try: #added
                        soup = get_soup(url)  # Get the soup object from the URL
                    except Exception as e:
                        print("Error in soup: ",e) #added
                    extracted_reviews = amazon_review(soup, url)  # Extract reviews from the soup
                   
                    if soup.find('div', {'class': 'a-section a-spacing-top-large a-text-center no-reviews-section'}):  
                            print('No review')  
                            found_reviews = False
                            break 
                    
                    #extend is to add a list of items 
                    if len(extracted_reviews) > 0:
                        all_reviews.extend(extracted_reviews) #extend is for appending a list 
                        print(f"Page {page} scraped {len(extracted_reviews)} reviews")
                    
                    # if (page == 1 and len(extracted_reviews) == 0):
                    #     print(f"Page {page} has no reviews, retry")
                    #     continue
                        
                    if soup.find('li', {'class': 'a-disabled a-last'}):  
                        print('No more pages left')  
                        found_reviews = False
                        break 
                    
                    if page >= 1 and len(extracted_reviews) == 0:
                        retry_attempts += 1
                        if retry_attempts == max_retry_attempts:
                            found_reviews = False
                            print(f"Page {page} has no reviews, moving to the next page")
                            break
                        else:
                            print(f"Page {page} has no reviews, retry")
                            continue 

                    
                            
                    else:
                        break  
        
                    

                except Exception as e:
                    # If any exception occurs, retry
                    retry_attempts += 1
                    if retry_attempts == max_retry_attempts:
                        break
                    else:
                        print(f"An error occurred, retrying")
                        continue  # Retry the loop
            else:
                # If all retry attempts failed, move to the next page
                continue
            
           



In [ ]:
print(type(all_reviews[0]))
print(len(all_reviews))
all_reviews


In [ ]:
from datetime import date 
pd.set_option('display.max_columns', None) #not doing anything
amazon2= pd.DataFrame(all_reviews) #convert list to df
amazon2['Retailer']="Amazon"
amazon2['scraping_date'] = pd.to_datetime(date.today())
amazon2['Review date'] = pd.to_datetime(amazon2['Review date'])
amazon2['Review title'] = amazon2['Review title'].str.extract(r'out of 5 stars\n(.*)') #this is regex
amazon2['HP Model Number'] = amazon2['Model'].str.extract(r'(\d+e?)')
amazon2['People_find_helpful'] = amazon2['People_find_helpful'].str.extract(r'(\d*) people found this helpful')

amazon_hp_combine = pd.merge(amazon2, df_amazon, on = "HP Model Number", how = "left" )
amazon_hp_combine['Review Model'] = amazon_hp_combine['HP Model'] 
columns_to_drop = [  
    'Model', 'HP Model Number', 'Comp Model number','HP Model'
]  
  
amazon_hp_combine = amazon_hp_combine.drop(columns_to_drop, axis = 1) 

amazon_hp_combine = amazon_hp_combine.drop_duplicates()
amazon_hp_combine 

In [ ]:
# # Save HP Amazon review if needed
# import datetime 
# date = datetime.date.today().strftime('%Y%m%d')
# amazon_hp_combine.to_csv(f'amazon_hp_{date}.csv')

## Comp review

In [ ]:
# Read url file
# path =  r"C:\Users\TaYu430\anaconda3\envs\webscrap\My Scripts\Star rating scrape URL and info - NPI.xlsx"
# path =  r"C:\Users\TaYu430\anaconda3\envs\webscrap\My Scripts\Star rating scrape URL and info - NPI.xlsx"
path =  r"Star rating scrape URL and info - NPI.xlsx"
sheets = 'Amazon'
amazon_url = pd.read_excel(path, sheet_name = sheets)
urls = amazon_url['Competitor URL'].to_list()


link_list = []
for value in urls:
    if value is not None and value not in link_list:
        link_list.append(value)
print(len(link_list)) #create a list of comp urls

In [ ]:
review_list_comp = []  

star = ['one','two','three','four','five'] 
max_retry_attempts = 20

for link in urls[0:1]:
    print(link)
    for y in star:
        found_reviews = True
        for page in range(1, 10):
            retry_attempts = 0
            while found_reviews is True:
                try:
                    print("===============")
                    url = f'{link}/ref=cm_cr_arp_d_viewopt_fmt?filterByStar={y}_star&pageNumber={page}&sortBy=recent&formatType=current_format'  
                    print(url)
                    print('Page:',page, f'{y} star')
                    soup = get_soup(url)  # Get the soup object from the URL
                    #print(soup)
                    #exit()
                    extracted_reviews = amazon_review(soup, url)  # Extract reviews from the soup
                   
                    if soup.find('div', {'class': 'a-section a-spacing-top-large a-text-center no-reviews-section'}):  
                            print('No review')  
                            found_reviews = False
                            break 
                    
                    if len(extracted_reviews) > 0:
                        review_list_comp.extend(extracted_reviews)
                        print(f"Page {page} scraped {len(extracted_reviews)} reviews")
                    
                    # if (page == 1 and len(extracted_reviews) == 0):
                    #     print(f"Page {page} has no reviews, retry")
                    #     continue
                        
                    if soup.find('li', {'class': 'a-disabled a-last'}):  
                        print('No more pages left')  
                        found_reviews = False
                        break 
                    
                    if page >= 1 and len(extracted_reviews) == 0:
                        retry_attempts += 1
                        if retry_attempts == max_retry_attempts:
                            found_reviews = False
                            print(f"Page {page} has no reviews, moving to the next page")
                            break
                        else:
                            print(f"Page {page} has no reviews, retry")
                            continue 

                    
                            
                    else:
                        break  
        
                    

                except Exception as e:
                    # If any exception occurs, retry
                    retry_attempts += 1
                    if retry_attempts == max_retry_attempts:
                        break
                    else:
                        print(f"An error occurred, retrying")
                        continue  # Retry the loop
            else:
                # If all retry attempts failed, move to the next page
                continue


In [ ]:
review_list_comp

In [ ]:
from datetime import date 
amazon_comp= pd.DataFrame(review_list_comp)
#debug: print amazon_comp to check before the Review date 
print(amazon_comp)
amazon_comp['Retailer']="Amazon"
amazon_comp['scraping_date'] =  pd.to_datetime(date.today())
amazon_comp['Review date'] = pd.to_datetime(amazon_comp['Review date']) #because there is no data
amazon_comp['Review title'] = amazon_comp['Review title'].str.extract(r'out of 5 stars\n(.*)')
amazon_comp['Comp Model number'] = amazon_comp['Model'].str.extract(r'(\d+)')
amazon_comp['People_find_helpful'] = amazon_comp['People_find_helpful'].str.extract(r'(\d*) people found this helpful')
amazon_comp = amazon_comp.drop_duplicates()

amazon_comp_combine = pd.merge(amazon_comp, df_amazon, on = 'Comp Model number', how = "left" )
amazon_comp_combine['Review Model'] = amazon_comp_combine['Comp Model'] 
columns_to_drop = [  
    'Model', 'HP Model Number', 'Comp Model number','HP Model'
]  

amazon_comp_combine = amazon_comp_combine.drop(columns_to_drop, axis = 1) 
amazon_comp_combine['Comp Model'] = ''
amazon_comp_combine['Segment'] = ''
amazon_comp_combine['HP Class'] = ''
amazon_comp_combine

## Combine comp and HP, concat with previous

In [ ]:
# add competitors below
# amazon_final = pd.concat([amazon_hp_combine, amazon_comp_combine], axis=0, ignore_index=True)  
amazon_final = amazon_hp_combine
amazon_final.drop_duplicates(inplace = True)
amazon_final['Review Content'] = amazon_final['Review Content'] .astype(str).apply(lambda x: re.sub(r'The media could not be loaded\.', '', x).strip())
amazon_final['Review Content'] = amazon_final['Review Content'].astype(str).apply(
    lambda x: re.sub(
        r'Video Player is loading\.Play VideoPlayMuteCurrent Time[\s\S]*?This is a modal window\.',
        '',
        x
    ).strip()
)

# amazon_final['Competitor_flag'] = amazon_final['Review Model'].apply(lambda x: 'No' if 'HP' in x else 'Yes')
amazon_final['Country'] = 'US'
amazon_final.sort_values(by = ['Review date'],ascending = False)

amazon_final_df= amazon_final.rename(columns={
    'HP Class': 'HP_Class',
    'Review Model': 'Review_Model',
    'Retailer': 'Retailer',
    'Comp Model': 'Comp_Model',
    'Review date': 'Review_Date',
    'Review name': 'Review_Name',
    'Review rating': 'Review_Rating',
    'Review title': 'Review_Title',
    'Review Content': 'Review_Content',
    'Verified Purchase or not': 'Verified_Purchase_Flag',
    'People_find_helpful': 'People_Find_Helpful',
    'Seeding or not': 'Seeding_Flag',
    'URL': 'URL',
    'scraping_date': 'Scraping_Date',
    'Segment': 'Segment',
    'Competitor_flag': 'Competitor_Flag',
    'Country': 'Country'
})

amazon_final_df['Review_Date'] = pd.to_datetime(amazon_final_df['Review_Date']).dt.date
amazon_final_df['Review_Rating'] = amazon_final_df['Review_Rating'].astype('int64')
amazon_final_df['People_Find_Helpful'] = amazon_final_df['People_Find_Helpful'].fillna(0).astype('int64')
amazon_final_df['Scraping_Date'] =  pd.to_datetime(amazon_final_df['Scraping_Date']).dt.date

amazon_final_df.reset_index(inplace = True,drop = True)
amazon_final_df.sort_values(['Review_Date'],ascending = False) 

## Read from SQL Server

In [ ]:
## Use encryption and pyodbc
import pyodbc

server = 'DB-Cluster3.ijp.sgp.rd.hpicorp.net'
database = 'STAR_Rating' 


# Establish connection using pyodbc
conn = pyodbc.connect(r'Driver={ODBC Driver 17 for SQL Server};Server='+server+';Database='+database+';Trusted_Connection=yes;Encrypt=yes;TrustServerCertificate=yes')

#Change model number accordingly
existing_rows_query = """
    SELECT *
    FROM STAR_Rating.dbo.Ink_web_reviews
    WHERE Retailer IN ('bestbuy')
    AND (Review_Model LIKE ('%2755e%')
    OR Review_Model LIKE ('%4155e%'))
"""

result_df = pd.read_sql_query(existing_rows_query, conn)

## Read from stackline data

In [ ]:
## For taccola
amazon2 = pd.read_csv(r"C:\Users\TaYu430\OneDrive - HP Inc\General - Core Team Laser & Ink\For Lip Kiat and Choon Chong\Web review\My Scripts\segment_Taccola_reviews_reviewTrends_Amazon_US_201609_202409.csv")

In [ ]:
amazon2 = amazon2.rename(columns={
    'Title':'Model',
     'Reviewer': 'Review name',
    'Review Stars': 'Review rating',
    'Review Text': 'Review Content',
    'Review Title': 'Review title',
    'Is Verified': 'Verified Purchase or not',
    'Reviews Url': 'URL',
     'Has Response':'Response_Text'
})

amazon2.columns

In [ ]:
from datetime import date 
pd.set_option('display.max_columns', None)

amazon2['Retailer']="Amazon"
amazon2['scraping_date'] = pd.to_datetime(date.today())
amazon2[ 'Review Date'] = pd.to_datetime(amazon2[ 'Review Date'])
amazon2['HP Model Number'] = amazon2['Model'].str.extract(r'(\d+e?)')


amazon_hp_combine = pd.merge(amazon2, df_amazon, on = "HP Model Number", how = "left" )
amazon_hp_combine['Review Model'] = amazon_hp_combine['HP Model'] 
columns_to_drop = [  
    'Model', 'HP Model Number', 'Comp Model number','HP Model'
]  
  
amazon_hp_combine = amazon_hp_combine.drop(columns_to_drop, axis = 1) 

amazon_hp_combine = amazon_hp_combine.drop_duplicates()
amazon_hp_combine 

In [ ]:
amazon_final = amazon_hp_combine
amazon_final.drop_duplicates(inplace = True)
amazon_final['Review Content'] = amazon_final['Review Content'] .astype(str).apply(lambda x: re.sub(r'The media could not be loaded\.', '', x).strip())
amazon_final['Review Content'] = amazon_final['Review Content'].astype(str).apply(
    lambda x: re.sub(
        r'Video Player is loading\.Play VideoPlayMuteCurrent Time[\s\S]*?This is a modal window\.',
        '',
        x
    ).strip()
)

amazon_final['Competitor_flag'] = amazon_final['Review Model'].apply(lambda x: 'No' if 'HP' in x else 'Yes')
amazon_final['Country'] = 'US'

amazon_final_df= amazon_final.rename(columns={
    'HP Class': 'HP_Class',
    'Review Model': 'Review_Model',
    'Retailer': 'Retailer',
    'Comp Model': 'Comp_Model',
    'Review Date': 'Review_Date',
    'Review name': 'Review_Name',
    'Review rating': 'Review_Rating',
    'Review title': 'Review_Title',
    'Review Content': 'Review_Content',
    'Verified Purchase or not': 'Verified_Purchase_Flag',
    'URL': 'URL',
    'scraping_date': 'Scraping_Date',
    'Segment': 'Segment',
    'Competitor_flag': 'Competitor_Flag',
    'Country': 'Country',
    'Review ID':'Review_ID'
})

amazon_final_df['Review_Date'] = pd.to_datetime(amazon_final_df['Review_Date']).dt.date
amazon_final_df['Review_Rating'] = amazon_final_df['Review_Rating'].astype('int64')
amazon_final_df['Scraping_Date'] =  pd.to_datetime(amazon_final_df['Scraping_Date']).dt.date

amazon_final_df.reset_index(inplace = True,drop = True)
amazon_final_df.sort_values(['Review_Date'],ascending = False) 
amazon_final_df.columns 
amazon_final_df.drop(columns=['Retailer ID', 'Retailer Name', 'Retailer SKU', 'UPC', 'Model Number',
       'Brand', 'Category', 'Subcategory', 'Week ID', 'Week Ending'],inplace = True)

In [ ]:
previous = amazon_final_df 
previous

In [ ]:
# result_df['Review_Date'] = pd.to_datetime(result_df['Review_Date'].dt.date)
def clean_text(text):
    text = str(text)

    # Remove non-English characters and punctuations
    cleaned_text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    # Remove extra whitespaces and convert to lowercase
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip().lower()
    english_words = re.findall(r'\b[a-z]+\b', cleaned_text)
    first_ten_words = ''.join(english_words[:10])
    return first_ten_words

# previous['FirstTenWords'] = previous['Review_Content'].fillna(0).apply(clean_text)

result_df['FirstTenWords'] = result_df['Review_Content'].fillna(0).apply(clean_text)
amazon_final_df['FirstTenWords'] = amazon_final_df['Review_Content'].fillna(0).apply(clean_text)

non_duplicated_df = amazon_final_df[(~amazon_final_df['Review_Date'].isin(result_df['Review_Date']))&
                                   (~amazon_final_df['FirstTenWords'].isin(result_df['FirstTenWords']))].drop_duplicates()
non_duplicated_df.drop(columns = ['FirstTenWords'], inplace = True)
non_duplicated_df.rename(columns = {'Aggregation':'Aggregation_Flag'}, inplace = True)
non_duplicated_df.sort_values(by = ['Review_Date'],ascending = True)
pd.set_option('display.max_rows', 20)
non_duplicated_df

In [ ]:
non_duplicated_df.drop(columns = 'Segment Name', inplace = True)

In [ ]:
## Save to SQL
server = 'DB-Cluster3.ijp.sgp.rd.hpicorp.net'
database = 'STAR_Rating' 

# Establish connection using pyodbc
conn = pyodbc.connect(r'Driver={ODBC Driver 17 for SQL Server};Server='+server+';Database='+database+';Trusted_Connection=yes;Encrypt=yes;TrustServerCertificate=yes')

dataframe = non_duplicated_df
table = "Ink_web_reviews"

chunk_size = 10000
total_rows = len(dataframe)
num_chunk = (total_rows + chunk_size - 1) // chunk_size

# Iterate over chunks and insert into the database
for i in range(num_chunk):
    start_index = i * chunk_size
    end_index = min((i + 1) * chunk_size, total_rows)  # Ensure end index doesn't exceed total rows
    chunk = dataframe.iloc[start_index:end_index]

    # Generate SQL INSERT statement
    sql_insert = f"INSERT INTO {table} ([{'], ['.join(chunk.columns)}]) VALUES ({', '.join(['?' for _ in chunk.columns])})"

    # Convert chunk DataFrame to list of tuples
    records = [tuple(row) for row in chunk.values]

    # Execute SQL INSERT statement
    cursor = conn.cursor()
    cursor.executemany(sql_insert, records)
    conn.commit()
    cursor.close()

    print(f"Chunk {i+1}/{num_chunk} saved to SQL.")
    
conn.close()


# Bestbuy

In [ ]:
from datetime import datetime
def get_review_bestbuy(url):
    extracted_reviews = []
    retry_count = 0
    header = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36 Edg/118.0.2088.61',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9',
        'Cache-Control': 'max-age=0',
        'Cookie': 'SID=5dd8d974-1010-4705-9db0-0091b9be90eb; bby_rdp=l; CTT=422cf77c62f741992b73b7eb194dd19d; intl_splash=false; intl_splash=false; vt=d36b7cc9-70f1-11ee-af65-0a4fc06e3e21; rxVisitor=169798943988975DRVD09AP9VHNKB488A7AMQ2ITCSNQ3; COM_TEST_FIX=2023-10-22T15%3A44%3A00.270Z; __gads=ID=6d604286666986e7:T=1697989449:RT=1697989449:S=ALNI_Mb_Z6tWUAT9d1smc0S2VYNtEXVnJQ; __gpi=UID=00000c6de2768122:T=1697989449:RT=1697989449:S=ALNI_MY8b96wWX_3ahxWOvsLcoQi2kpHIA; s_ecid=MCMID%7C51499273735922173403879288947271341352; AMCVS_F6301253512D2BDB0A490D45%40AdobeOrg=1; dtCookie=v_4_srv_5_sn_UKGS61LHKE95F58CKCJ5JTTUHNJV2N7D_app-3A1b02c17e3de73d2a_1_ol_0_perc_100000_mul_1; _cs_mk=0.5500628905410729_1697989446664; s_cc=true; AMCV_F6301253512D2BDB0A490D45%40AdobeOrg=1585540135%7CMCMID%7C51499273735922173403879288947271341352%7CMCAID%7CNONE%7CMCOPTOUT-1697996646s%7CNONE%7CMCAAMLH-1698594246%7C3%7CMCAAMB-1698594246%7Cj8Odv6LonN4r3an7LhD3WZrU1bUpAkFkkiY1ncBR96t2PTI%7CMCCIDH%7C1907712470%7CvVersion%7C4.4.0; aam_uuid=56460070521806806704392296716542884874; locDestZip=96939; locStoreId=1760; sc-location-v2=%7B%22meta%22%3A%7B%22CreatedAt%22%3A%222023-10-22T15%3A44%3A06.975Z%22%2C%22ModifiedAt%22%3A%222023-10-22T15%3A44%3A07.381Z%22%2C%22ExpiresAt%22%3A%222024-10-21T15%3A44%3A07.381Z%22%7D%2C%22value%22%3A%22%7B%5C%22physical%5C%22%3A%7B%5C%22zipCode%5C%22%3A%5C%2296939%5C%22%2C%5C%22source%5C%22%3A%5C%22G%5C%22%2C%5C%22captureTime%5C%22%3A%5C%222023-10-22T15%3A44%3A06.975Z%5C%22%7D%2C%5C%22destination%5C%22%3A%7B%5C%22zipCode%5C%22%3A%5C%2296939%5C%22%7D%2C%5C%22store%5C%22%3A%7B%5C%22storeId%5C%22%3A1760%2C%5C%22zipCode%5C%22%3A%5C%2299504%5C%22%2C%5C%22storeHydratedCaptureTime%5C%22%3A%5C%222023-10-22T15%3A44%3A07.380Z%5C%22%7D%7D%22%7D; __gsas=ID=43dc00dcffeab34e:T=1697989465:RT=1697989465:S=ALNI_MYLHkniZY8kqCiAFOeNu1jnR4mz0w; dtSa=-; cto_bundle=2D7FnF9ZMHJPQlFCbkdTMktUSFREZ2pVJTJGajJMRFFsd2lINnRNRkZxY0dFU1lqJTJCN0glMkZMU0FqRTR0UyUyRmZRa1FscDdyV0tQUTNZdzVBM1g2WkJHUENTUEdlaGtUdWtiZWU4allOYlc2dyUyRm1VeiUyRlVBZVZkdVRmSFElMkJZQ0ExRk9mZzZNV1VNd1ZYSXZ5RWZSeUFQdkJXZ3VxZzZJZyUzRCUzRA; blue-assist-banner-shown=true; _cs_c=1; _gcl_au=1.1.1372174147.1697989479; dtLatC=1; _abck=2025C1ED2DAE1BA19B91708C91F51C0F~0~YAAQHLQRYGhMakWLAQAAjTYLWgqyxn7G2wIoFoVC+4nrsT1cxJIaO1O5ytS58DrifnksxvYxu7oOIuZmBDszkeEGLUk/7ekIvtGFO7u2yogmIcW17juPvPSDc1XdGYIVbijt6PbXvKVWeAB+8ZIF6voDPAwIN8H+QKpGl7va06mSquCsIXDORvQ1fz6MaHlKajkG/g9N8gGFlrsBxnMpRA0vk4b7Xv9obYx0wvld8KvntBNHHmpIs0djlSe17djNQz57X3JJHstt9/StCh7Jo00MTiV93eKEGVBoMzoq4+PxnTdsrKg5PkI1bneUzJMSGuV43ZaXWfbm7uJ5sVfxdvHl0uQOQUh7ClSLpjFxe7sR9F6ZRsJ1uTIjK2Ab7WfvjLZd5C8V7/qZhg/oMP3pF0Dt09LThXO7tonFOvt8UhAETsU0Hw6+K/m4mS0wH46V+5rfa+qmNcM=~-1~-1~-1; bm_sz=DC447A131B862AC781959292B401C641~YAAQHLQRYGlMakWLAQAAjTYLWhWe27kjreKQmsKd+a2iqr9yFDHU3maKKvHTexZicnoFjIsx0OiZ03lAbfGOl2IZo7UNsbeBjNT3emSu3sSR0HUl0ddFd8LjnFGqQISSIw7upSTqhbE/Ccdgbo842X0fWkxXLQCXe7eIC5cgVWU1GMRdWc34I/WgCiVwaRV2v6j3I93rIKuMA5dYvCv2yQykBKCPPN4sbyl8TEvfZ+XgvWuziGVpb4G+3OBohzrz8/j7ZnhXQ1U0WZARKye28p1zLuSDfDk4mInPZlvumI5oeG13Z+CjpYEKf7D5iAjzcWRGlsQ32gejCk7aPI6RC1dkVBh/DL00bGUor4wdKjruVwQNpz0v3hop17nvb4BKkQIqQAfEL6zMaGHLj9ycBq93U+2b2AXxNlcKMzEYuQ0cL/PuJIlwGBiqjQ==~4601667~3422276; dtPC=5$589618119_27h-vVFAAHBSMMCTLEHFRWVQLIOPDRRSURPFR-0e0; rxvt=1698031868366|1698030068366; _cs_id=3fe9d270-9876-ad3a-cae4-0084c344a27c.1697989478.5.1698030169.1698030169.1645469968.1732153478774; _cs_s=1.0.0.1698031970357; c2=pdp%3A%20ratingsreviews',  # Replace with the actual Cookie
        'Downlink': '10',
        'Dpr': '1',
        'Referer': url,
        'Sec-Ch-Ua': '"Google Chrome";v="117", "Not;A=Brand";v="8", "Chromium";v="117"',
        'Sec-Ch-Ua-Mobile': '?0',
        'Sec-Ch-Ua-Platform': '"Windows"',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'cross-site',
        'Sec-Fetch-User': '?1',
        'Upgrade-Insecure-Requests': '1'
    }

    response = requests.get(url, headers=header)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup 

def bestbuy_review(soup, url):    
    bestbuy = {}
    bestbuy_reviews = []  
    Model = soup.find("h1", {'class':"heading-5 v-fw-regular"})
    if not Model:
        Model = soup.find("h2", {"class": "heading-6 product-title mb-100"})
    Model = Model.text if Model else None
    
        
    npi = soup.find('span',{'class':'c-reviews order-2'} ).text
    review_session = soup.find_all("div", {"class": "review-item-content col-xs-12 col-md-9"})
    if review_session:
        for item in review_session:    
            bestbuy = {    
                 'Model':Model,
                # 'Review date': item.find("div", {"class": "posted-date-ownership disclaimer v-m-right-xxs"}).text.replace('Posted','')  
                'URL':url 
            }
            try:    
                bestbuy['Review title']  = item.find("h4", {"class": "c-section-title review-title heading-5 v-fw-medium"}).text  
            except AttributeError:    
                bestbuy['Review title']  = None

            try:
                bestbuy['Review_Name']  = item.find("div", {"class": "ugc-author v-fw-medium body-copy-lg"}).text  
            except AttributeError:    
                bestbuy['Review_Name']  = None
                

            try:    
                bestbuy['Review rating']  = item.find("div", {"class": "c-ratings-reviews flex c-ratings-reviews-small align-items-center gap-50"}).text.replace(' out of 5 stars','').replace('Rated ','')  
            except AttributeError:    
                bestbuy['Review rating']  = None

            review_date_element = item.find("time", {"class": "submission-date"})
            if review_date_element:
                review_date_string = review_date_element['title']
                review_date_datetime = datetime.strptime(review_date_string, '%b %d, %Y %I:%M %p')
                formatted_review_date = review_date_datetime.strftime('%Y-%m-%d')
                bestbuy['Review_Date'] = formatted_review_date
            else:
                bestbuy['Review_Date'] = ""

            try:    
                bestbuy['Review promotion']  = item.find("div", {"class": "body-copy-sm pt-50"}).text
            except AttributeError:    
                bestbuy['Review promotion']  = None

            try:    
                bestbuy['Review aggregation']  = item.find("p", {"class": "body-copy ugc-related-product"}).text
            except AttributeError:    
                bestbuy['Review aggregation']  = None

            try:    
                bestbuy['Review Content']  = item.find("div", {"class": "ugc-review-body"}).text  
            except AttributeError:    
                bestbuy['Review Content']  = None

            try:    
                bestbuy['Review Recommendation']  = item.find("div", {"class": "ugc-recommendation"}).text  
            except AttributeError:    
                bestbuy['Review Recommendation']  = None

            try:    
                network_badge  = item.find("div", {"class": "ugc-network-badge"})
                if network_badge:
                    bestbuy['Seeding or not'] = network_badge.get("data-track")
                else:
                    bestbuy['Seeding or not']  = ""
            except AttributeError:    
                bestbuy['Seeding or not']  = ""

            try:    
                bestbuy['People_find_helpful']  = item.find("button", {"data-track": "Helpful"}).text
            except AttributeError:    
                bestbuy['People_find_helpful']  = None

            try:    
                bestbuy['People_find_unhelpful']  = item.find("button", {"data-track": "Unhelpful"}).text
            except AttributeError:    
                bestbuy['People_find_unhelpful']  = None


            bestbuy_reviews.append(bestbuy)    
        
    
  
    return npi, bestbuy_reviews 

## Best buy hp

In [ ]:
urls = [

## Manhatten
#     'https://www.bestbuy.com/site/reviews/hp-officejet-pro-9015e-wireless-all-in-one-inkjet-printer-with-6-months-of-instant-ink-included-with-hp-white/6450667?variant=A',
# 'https://www.bestbuy.com/site/reviews/hp-officejet-pro-9025e-wireless-all-in-one-inkjet-printer-with-6-months-of-instant-ink-included-with-hp-white/6450665?variant=A'
# 'https://www.bestbuy.com/site/reviews/hp-smart-tank-5101-wireless-all-in-one-supertank-inkjet-printer-with-up-to-2-years-of-ink-included-white/6527565?variant=A'


## Taccola
    'https://www.bestbuy.com/site/reviews/hp-deskjet-2755e-wireless-inkjet-printer-with-3-months-of-instant-ink-included-with-hp-white/6454282?variant=A',
   'https://www.bestbuy.com/site/reviews/hp-deskjet-4155e-wireless-all-in-one-inkjet-printer-with-3-months-of-instant-ink-included-with-hp-white/6454283?variant=A'
]

In [ ]:
bestbuy_reviews = []

for link in urls:
    print(link)
    should_continue = True
    for x in range(1, 25):
        if not should_continue:
            break
        while True:
            url = f'{link}&page={x}&sort=MOST_RECENT'
            try:
                soup = get_review_bestbuy(url)
                npi, reviews = bestbuy_review(soup, url)
                if npi == 'Be the first to write a review':
                    should_continue = False
                print(f'Extracted reviews on page {x}: {len(reviews)}')
                bestbuy_reviews.extend(reviews)

                next_page_link = soup.find("a", {"aria-disabled": "true"})  # Note: Use lowercase "true" for attribute value
                if x > 1 and next_page_link and next_page_link.get("aria-disabled") == "true":
                    should_continue = False
                    print('No more pages left')
                    break

                if len(reviews) < 20:
                    should_continue = False
                    print('Only 1 page')
                    break
                else:
                    break 
            except Exception as e:
                print(f"Error encountered: {e}. Retrying in 3 seconds...")
                time.sleep(3)


In [ ]:
from datetime import date  
pd.set_option('display.max_columns', None)
review = pd.DataFrame(bestbuy_reviews)
review['Retailer']="bestbuy"
review['scraping_date'] = pd.to_datetime(date.today())

review['HP Model Number'] = review['Model'].str.extract(r'(\d+e*)')

hp_combine = pd.merge(review, df_amazon, on = "HP Model Number", how = "left" )


In [ ]:
hp_combine['Review Model'] = hp_combine['HP Model'] 
# hp_combine['People_find_helpful'] = hp_combine['People_find_helpful'].fillna("").str.extract(r'(\d+)').astype(int)



columns_to_drop = [  
    'Model', 'HP Model Number', 'Comp Model number','HP Model'
]  
  
hp_combine_bestbuy = hp_combine.drop(columns_to_drop, axis = 1) 

hp_combine_bestbuy = hp_combine_bestbuy.drop_duplicates()




In [ ]:
# bestbuy_final = pd.concat([hp_combine_bestbuy, comp_combine_bestbuy], axis=0, ignore_index=True)  
bestbuy_final = hp_combine_bestbuy
bestbuy_final.drop_duplicates(inplace = True)

bestbuy_final = bestbuy_final.sort_values(by = ['Review Model', 'Review title', 'Review Content', 'scraping_date'])

bestbuy_final['Competitor_Flag'] = bestbuy_final['Review Model'].apply(lambda x: 'No' if 'HP' in x else 'Yes')
bestbuy_final['Country'] = 'US'

bestbuy_final_version= bestbuy_final.rename(columns={
    'HP Class': 'HP_Class',
    'Review Model': 'Review_Model',
    'Retailer': 'Retailer',
    'Comp Model': 'Comp_Model',
    'Review_Date': 'Review_Date',
    'Review name': 'Review_Name',
    'Review rating': 'Review_Rating',
    'Review title': 'Review_Title',
    'Review Content': 'Review_Content',
    'Seeding or not': 'Seeding_Flag',
    'People_find_helpful': 'People_Find_Helpful',
    'URL': 'URL',
    'scraping_date': 'Scraping_Date',
    'Review promotion': 'Promotion_Flag',
    'Review aggregation': 'Aggregation_Flag'
})

bestbuy_final_version.drop(columns = ['Review Recommendation',  'People_find_unhelpful'],inplace = True)



In [ ]:
Final_review = pd.concat([review_template, bestbuy_final_version], ignore_index = True)
len(Final_review.columns)

In [ ]:
Final_review['Review_Date'] = pd.to_datetime(Final_review['Review_Date']).dt.date
Final_review['Review_Rating'] = Final_review['Review_Rating'].astype('int64')
Final_review['People_Find_Helpful'] = Final_review['People_Find_Helpful'].str.extract(r'\((\d+)\)')
Final_review['Scraping_Date'] =  pd.to_datetime(Final_review['Scraping_Date']).dt.date
Final_review

## Save to SQL

In [ ]:
## Use encryption and pyodbc
import pyodbc

server = 'DB-Cluster3.ijp.sgp.rd.hpicorp.net'
database = 'STAR_Rating' 

# Establish connection using pyodbc
conn = pyodbc.connect(r'Driver={ODBC Driver 17 for SQL Server};Server='+server+';Database='+database+';Trusted_Connection=yes;Encrypt=yes;TrustServerCertificate=yes')

#Change retailer and model number accordingly
existing_rows_query = """
    SELECT *
    FROM STAR_Rating.dbo.Ink_web_reviews
    WHERE Retailer IN ('bestbuy')
    AND (Review_Model LIKE ('%2755e%')
    OR Review_Model LIKE ('%4155e%'))
"""

result_df = pd.read_sql_query(existing_rows_query, conn)

In [ ]:
result_df

In [ ]:
# result_df['Review_Date'] = pd.to_datetime(result_df['Review_Date'].dt.date)
def clean_text(text):
    text = str(text)

    # Remove non-English characters and punctuations
    cleaned_text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    # Remove extra whitespaces and convert to lowercase
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip().lower()
    english_words = re.findall(r'\b[a-z]+\b', cleaned_text)
    first_ten_words = ''.join(english_words[:10])
    return first_ten_words

Final_review['FirstTenWords'] = Final_review['Review_Content'].fillna(0).apply(clean_text)

result_df['FirstTenWords'] = result_df['Review_Content'].fillna(0).apply(clean_text)


non_duplicated_df = Final_review[(~Final_review['Review_Date'].isin(result_df['Review_Date']))&
                                   (~Final_review['FirstTenWords'].isin(result_df['FirstTenWords']))].drop_duplicates()
non_duplicated_df.drop(columns = ['FirstTenWords'], inplace = True)
non_duplicated_df.rename(columns = {'Aggregation':'Aggregation_Flag'}, inplace = True)
non_duplicated_df.sort_values(by = ['Review_Date'],ascending = True)
pd.set_option('display.max_rows', 20)
non_duplicated_df



In [ ]:
non_duplicated_df['People_Find_Helpful'] = non_duplicated_df['People_Find_Helpful'].fillna(0).astype(int)
non_duplicated_df.drop(columns = 'Syndicated_Source', inplace = True)

In [ ]:
non_duplicated_df.info()

In [ ]:
from sqlalchemy import create_engine, text
 
server = 'DB-Cluster3.ijp.sgp.rd.hpicorp.net'
database = 'STAR_Rating'
schema = 'dbo'
driver = 'ODBC Driver 17 for SQL Server'

dataframe = non_duplicated_df
table = "Ink_web_reviews"

engine = create_engine(f"mssql+pyodbc://{server}/{database}?driver={driver}", echo=True)
 
chunk_size = 10000
total_rows = len(dataframe)
num_chunk = (total_rows + chunk_size - 1) // chunk_size

for i in range(num_chunk):
    start_index = i * chunk_size
    end_index = (i + 1) * chunk_size
    chunk = dataframe.iloc[start_index:end_index]
    
    chunk.to_sql(table, engine, index=False, if_exists="append", schema="dbo")
    print(f"Chunk {i+1}/{num_chunk} saved to SQL.")

# Staple

In [ ]:
#new
def staple_review(url, sku, max_pages):
    base_url = 'https://www.staples.com/sdc/ptd/api/reviewProxy/getReviews'
    
    headers = {
        'Content-Type': 'application/json',
        'Referer': url
    }

    all_data = []  # List to store data from all pages

    for page in range(1, max_pages + 1):
        params = {
            'tenantType': 'StaplesDotCom',
            'sku': sku,
            'offset': (page - 1) * 20,
            'limit': 20,
            'includeRelated': 'false',
            'relatedOnly': 'false',
            'includeRatingOnlyReviews': 'false',
            'sortBy': 'date',
            'sortOrder': 'desc'
        }

        # Make the GET request
        response = requests.get(base_url, params=params, headers=headers)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the JSON content
            data = response.json()
            reviews = data.get('reviewList', {}).get('reviews', [])
            if reviews:
                df = pd.DataFrame(reviews)
                all_data.append(df)
        else:
            print(f"Failed to retrieve data for page {page}. Status code: {response.status_code}")

    # Concatenate all dataframes into a single dataframe
    if all_data:
        result_df = pd.concat(all_data, ignore_index=True)
        return result_df
    else:
        return pd.DataFrame()

def max_pages(sku, url):
    base_url = 'https://www.staples.com/sdc/ptd/api/reviewProxy/getReviews'
    
    headers = {
        'Content-Type': 'application/json',
        'Referer': url
    }

    params = {
        'tenantType': 'StaplesDotCom',
        'sku': sku,
        'offset': 0,
        'limit': 20,
        'includeRelated': 'false',
        'relatedOnly': 'false',
        'includeRatingOnlyReviews': 'false',
        'sortBy': 'date',
        'sortOrder': 'desc'
    }

    # Make the GET request
    response = requests.get(base_url, params=params, headers=headers)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the JSON content
        data = response.json()
        # print(data)
        total_reviews = data.get('reviewList', {}).get('total', 0)
        max_pages = math.ceil(total_reviews / 20)
        return max_pages
    else:
        print(f"Failed to retrieve data. Status code: {response.status_code}")
        return 0

def extract_sku_from_url(url):
    # Use regular expression to extract numeric value from the end of the URL
    match = re.search(r'/(\d+)$', url)
    if match:
        return match.group(1)
    return None

# Main scraping process
staples_df_hp = pd.DataFrame()

# Define URLs
urls = ['https://www.staples.com/ptd/review/24455371']

for url in urls:
    sku = extract_sku_from_url(url)
    print('Get reviews from', url)
    total_pages = max_pages(sku, url)
    print('Total pages:', total_pages)
    
    if total_pages > 0:
        data = staple_review(url, sku, total_pages)
        print('Total Reviews scraped:', len(data))
        if not data.empty:
            staples_df_hp = pd.concat([staples_df_hp, data], axis=0)

# Post-processing and data cleanup
def extract_source_name(user_dict):
    if isinstance(user_dict, dict):
        return user_dict.get('sourceName', '')
    else:
        return ''

if 'syndication' in staples_df_hp.columns:    
    staples_df_hp['syndication'] = staples_df_hp['syndication'].apply(extract_source_name)

staple_final = staples_df_hp[staples_df_hp['published'] == True]
staple_final['Model'] = staple_final['catalogItems'].apply(lambda x: x[0]['title'] if x else None)

if 'syndication' in staple_final.columns:
    staple_final = staple_final[['id', 'dateCreated', 'title', 'text', 'rating', 'user', 'syndication', 'incentivized', 'Model']]
else:
    staple_final = staple_final[['id', 'dateCreated', 'title', 'text', 'rating', 'user', 'incentivized', 'Model']]

def tidy_up_user(user):
    if isinstance(user, dict) and 'nickName' in user:
        return user['nickName']
    else:
        return 'blank'

staple_final['user'] = staple_final['user'].apply(tidy_up_user)

# Additional data processing
pd.set_option('display.max_columns', None)
staple_final['Retailer'] = "Staples"
staple_final['scraping_date'] = pd.to_datetime(date.today())
staple_final['HP Model Number'] = staple_final['Model'].str.extract(r'(\d+e?)')

# Merge and data clean-up
staple_hp_combine = pd.merge(staple_final, df_amazon, on="HP Model Number", how="left")
staple_hp_combine['Review Model'] = staple_hp_combine['HP Model'] 

columns_to_drop = ['Model', 'HP Model Number', 'Comp Model number', 'HP Model', 'id']
staple_hp_combine.drop(columns_to_drop, axis=1, inplace=True)

staple_hp_combine['Country'] = 'US'

column_mapping = {
    'HP Class': 'HP_Class',
    'dateCreated': 'Review_Date',
    'text': 'Review_Content',
    'title': 'Review_Title',
    'incentivized': 'Seeding_Flag',
    'user': 'Review_Name',
    'syndication': 'Syndicated_Source',
    'rating': 'Review_Rating',
    'Retailer': 'Retailer',
    'Comp Model': 'Comp_Model',
    'scraping_date': 'Scraping_Date',
    'Review Model': 'Review_Model'
}

# 'HP Class': 'HP_Class',
#     'Review Model': 'Review_Model',
#     'Retailer': 'Retailer',
#     'Comp Model': 'Comp_Model',
#     'Review_Date': 'Review_Date',
#     'Review name': 'Review_Name',
#     'Review rating': 'Review_Rating',
#     'Review title': 'Review_Title',
#     'Review Content': 'Review_Content',
#     'Seeding or not': 'Seeding_Flag',
#     'People_find_helpful': 'People_Find_Helpful',
#     'URL': 'URL',
#     'scraping_date': 'Scraping_Date',
#     'Review promotion': 'Promotion_Flag',
#     'Review aggregation': 'Aggregation_Flag'


# Rename columns
staple_final = staple_hp_combine.rename(columns=column_mapping)
staple_final['Competitor_Flag'] = staple_final['Review_Model'].apply(lambda x: 'No' if 'HP' in x else 'Yes')
staple_final.drop_duplicates(inplace=True)
staple_final['Review_Date'] = pd.to_datetime(staple_final['Review_Date']).dt.date


In [ ]:
Final_review = staple_final 
Final_review 

## Combine

In [ ]:
pattern = r'(\w+ \d{1,2}, \d{4})'
Final_review['Review_Date'] = pd.to_datetime(Final_review['Review_Date']).dt.date
Final_review['Review_Rating'] = Final_review['Review_Rating'].astype('int64')
Final_review['Scraping_Date'] =  pd.to_datetime(Final_review['Scraping_Date']).dt.date
Final_review.info()

## Save to SQL

In [ ]:
# Query previous review
from sqlalchemy import create_engine

server = 'DB-Cluster3.ijp.sgp.rd.hpicorp.net'
database = 'STAR_Rating'
schema = 'dbo'
driver = 'ODBC Driver 17 for SQL Server'

# dataframe = amazon_final_df
table = "Ink_web_reviews"

engine = create_engine(f"mssql+pyodbc://{server}/{database}?driver={driver}", echo=True)


existing_rows_query = f"""
    SELECT *
    FROM {schema}.{table}
    WHERE Retailer in ('staples')
    AND Review_Model in ('HP OfficeJet Pro 9015e',
'HP OfficeJet Pro 9025e'
)
"""
result_df = pd.read_sql_query(existing_rows_query, engine)

def clean_text(text):
    text = str(text)

    # Remove non-English characters and punctuations
    cleaned_text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    # Remove extra whitespaces and convert to lowercase
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip().lower()
    english_words = re.findall(r'\b[a-z]+\b', cleaned_text)
    first_ten_words = ''.join(english_words[:10])
    return first_ten_words

previous['FirstTenWords'] = previous['Review_Content'].fillna(0).apply(clean_text)
result_df['FirstTenWords'] = result_df['Review_Content'].fillna(0).apply(clean_text)
Final_review['FirstTenWords'] = Final_review['Review_Content'].fillna(0).apply(clean_text)


non_duplicated_df = Final_review[
                                   (~Final_review['FirstTenWords'].isin(result_df['FirstTenWords']))].drop_duplicates()
# non_duplicated_df.drop(columns = ['FirstTenWords'], inplace = True)


In [ ]:
non_duplicated_df 

In [ ]:
import datetime
filter_date = datetime.date(2023, 12, 14)
non_duplicated_df['Review_Date'] = pd.to_datetime(non_duplicated_df['Review_Date']).dt.date

# Filter the DataFrame to keep only rows after '2023-12-14'
non_duplicated_df = non_duplicated_df[non_duplicated_df['Review_Date'] > filter_date]

non_duplicated_df = non_duplicated_df.iloc[:, :-1] ##New Line

non_duplicated_df

## check for error

In [ ]:
from sqlalchemy import create_engine, text
 
server = 'DB-Cluster3.ijp.sgp.rd.hpicorp.net'
database = 'STAR_Rating'
schema = 'dbo'
driver = 'ODBC Driver 17 for SQL Server'

dataframe = non_duplicated_df
table = "Ink_web_reviews"
print(dataframe)

# dataframe.drop(columns = ['FirstTenWords'], inplace = True)

# print("new dataframe:", dataframe)

engine = create_engine(f"mssql+pyodbc://{server}/{database}?driver={driver}", echo=True)
 
chunk_size = 10000
total_rows = len(dataframe)
num_chunk = (total_rows + chunk_size - 1) // chunk_size

for i in range(num_chunk):
    start_index = i * chunk_size
    end_index = (i + 1) * chunk_size
    chunk = dataframe.iloc[start_index:end_index]
    
    chunk.to_sql(table, engine, index=False, if_exists="append", schema="dbo")
    print(f"Chunk {i+1}/{num_chunk} saved to SQL.")

# Walmart

In [ ]:
import random
User_Agent = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36 Edg/120.0.0.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36 Edg/119.0.0.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
   ,'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:109.0) Gecko/20100101 Firefox/109.0',
    'Mozilla/5.0 (Linux; Android 11; SAMSUNG SM-G973U) AppleWebKit/537.36 (KHTML, like Gecko)',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'Mozilla/5.0 (X11; U; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.5399.183 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:109.0) Gecko/20100101 Firefox/113.0'
]
              

In [ ]:
cookie = [
    'ACID=e743918f-9c01-4185-9889-01b383f39a46; hasACID=true; _m=9; locGuestData=eyJpbnRlbnQiOiJTSElQUElORyIsImlzRXhwbGljaXQiOmZhbHNlLCJzdG9yZUludGVudCI6IlBJQ0tVUCIsIm1lcmdlRmxhZyI6ZmFsc2UsImlzRGVmYXVsdGVkIjp0cnVlLCJwaWNrdXAiOnsibm9kZUlkIjoiMzA4MSIsInRpbWVzdGFtcCI6MTcwMDI5Mjg0MjMwNCwic2VsZWN0aW9uVHlwZSI6IkRFRkFVTFRFRCJ9LCJzaGlwcGluZ0FkZHJlc3MiOnsidGltZXN0YW1wIjoxNzAwMjkyODQyMzA0LCJ0eXBlIjoicGFydGlhbC1sb2NhdGlvbiIsImdpZnRBZGRyZXNzIjpmYWxzZSwicG9zdGFsQ29kZSI6Ijk1ODI5IiwiY2l0eSI6IlNhY3JhbWVudG8iLCJzdGF0ZSI6IkNBIiwiZGVsaXZlcnlTdG9yZUxpc3QiOlt7Im5vZGVJZCI6IjMwODEiLCJ0eXBlIjoiREVMSVZFUlkiLCJ0aW1lc3RhbXAiOjE3MDAyOTI4NDIzMDMsInNlbGVjdGlvblR5cGUiOiJERUZBVUxURUQiLCJzZWxlY3Rpb25Tb3VyY2UiOm51bGx9XX0sInBvc3RhbENvZGUiOnsidGltZXN0YW1wIjoxNzAwMjkyODQyMzA0LCJiYXNlIjoiOTU4MjkifSwibXAiOltdLCJ2YWxpZGF0ZUtleSI6InByb2Q6djI6ZTc0MzkxOGYtOWMwMS00MTg1LTk4ODktMDFiMzgzZjM5YTQ2In0%3D; vtc=TPvkWCN79GksgVjpd8lQp4; btc=TPvkWCN79GksgVjpd8lQp4; bsc=XjMXQ-W4dVqYGkP1JSNWEs; _pxvid=d8404a5f-85e4-11ee-a839-f5e2c99825fc; pxcts=dd47a175-85e4-11ee-b2aa-79f32486f419; _tap_path=/rum.gif; _tap-criteo=1700292852331:1700292852725:1; _tap-Ted=1700292852724:1700292852725:1; _tap-lrV=1700292862974:1700292862975:1; _tap-lrB=1700292878531:1700292878532:1; _tap-appnexus=1700292879419:1700292879718:1; _gcl_au=1.1.258469864.1700294686; bstc=XgTuQhQdHEhnMqLxEbRYgw; mobileweb=0; xpth=x-o-mart%2BB2C~x-o-mverified%2Bfalse; xpa=; ak_bmsc=928D3BE7B227834B8EF4992840D6AFA1~000000000000000000000000000000~YAAQT0Dfb2Hq79qLAQAAWF0E7xW+cNY2y9+fVR2/0QrvmtyiqK/i/uEwVaXReQYp0Z0y6FV2m0EQCZomlg6PikSMdPE5Yx+Al074WilmM5mGxgz2kYLAXQy0aPKfmDgo9ohHooJNwq7VcMShtmCmJ1ARCqwfhpgMsbNFrR7LgHL3FRZJhrg9alhKuKLZRfToimKXZ0sKxSm5rLcO6L57KGoWlZX3NmA0ddatFrENafTH0mQVlKWpVPKHtgOZZh4EsGHOO90Z1fy0P2hIJNmpHpoDLk1w+0tBu9Upo+8L5KPS0K1LcuVDuaXEx4+n6o2fMg+HrkC1AF5T7cKLiLDZlbL/C5kSesKN3Rp2RaJjt8YzY2QTkaxrmX9TYd6omcLPWSqHLvjJK0/gyfE=; xptc=assortmentStoreId%2B3081; xpm=3%2B1700522123%2BTPvkWCN79GksgVjpd8lQp4~%2B0; b30msc=XgTuQhQdHEhnMqLxEbRYgw; _tap-li=1700522132702:0:2; _uetsid=0cce940087ff11eea88661d0630688d7; _uetvid=237553a085e911ee9eaec99bfa867851; auth=MTAyOTYyMDE4TzUL0tBWqaWlPLiIWSvGIIqP44I9XeKX8N5bcOOhCkNGFqMJTLhMlajVoxZh6%2F64P%2Bx3aAqHkDo7tLTswFgLqhE9gq7sWtCKMSvMMtAlsDnj17kGZ7Mu4K7gp6blyLzB767wuZloTfhm7Wk2KcjygsAEeU%2BeKCMhfP9XV060SY%2Fspww18DSfg4loIXetO33HWWxCKdp%2B8UHdguRD9DC%2FlTyW2FeTzNUdxbN2aHvb8W0UMk70P8glgOEpLOprhDfMDCcb9mgycy9jtT1uIyOBHWs0k33oHkBmchRaU9fj5kF7pZ6JaDMzmiWlGlRQ4nUMw5XFK3QDKKcB%2BPe5gKPHMTJomOWP4NHaOZmjOE06S78R4yUE7XpP0usJVgwSa5Hg5dejwrW41QOfpHzdmIzkekjyrOXbKKhH072NS%2FW0j%2FU%3D; locDataV3=eyJpc0RlZmF1bHRlZCI6dHJ1ZSwiaXNFeHBsaWNpdCI6ZmFsc2UsImludGVudCI6IlNISVBQSU5HIiwicGlja3VwIjpbeyJidUlkIjoiMCIsIm5vZGVJZCI6IjMwODEiLCJkaXNwbGF5TmFtZSI6IlNhY3JhbWVudG8gU3VwZXJjZW50ZXIiLCJub2RlVHlwZSI6IlNUT1JFIiwiYWRkcmVzcyI6eyJwb3N0YWxDb2RlIjoiOTU4MjkiLCJhZGRyZXNzTGluZTEiOiI4OTE1IEdlcmJlciBSb2FkIiwiY2l0eSI6IlNhY3JhbWVudG8iLCJzdGF0ZSI6IkNBIiwiY291bnRyeSI6IlVTIiwicG9zdGFsQ29kZTkiOiI5NTgyOS0wMDAwIn0sImdlb1BvaW50Ijp7ImxhdGl0dWRlIjozOC40ODI2NzcsImxvbmdpdHVkZSI6LTEyMS4zNjkwMjZ9LCJpc0dsYXNzRW5hYmxlZCI6dHJ1ZSwic2NoZWR1bGVkRW5hYmxlZCI6dHJ1ZSwidW5TY2hlZHVsZWRFbmFibGVkIjp0cnVlLCJodWJOb2RlSWQiOiIzMDgxIiwic3RvcmVIcnMiOiIwNjowMC0yMzowMCIsInN1cHBvcnRlZEFjY2Vzc1R5cGVzIjpbIlBJQ0tVUF9DVVJCU0lERSIsIlBJQ0tVUF9JTlNUT1JFIl0sInNlbGVjdGlvblR5cGUiOiJERUZBVUxURUQifV0sInNoaXBwaW5nQWRkcmVzcyI6eyJsYXRpdHVkZSI6MzguNDc0NSwibG9uZ2l0dWRlIjotMTIxLjM0MzgsInBvc3RhbENvZGUiOiI5NTgyOSIsImNpdHkiOiJTYWNyYW1lbnRvIiwic3RhdGUiOiJDQSIsImNvdW50cnlDb2RlIjoiVVNBIiwiZ2lmdEFkZHJlc3MiOmZhbHNlLCJ0aW1lWm9uZSI6IkFtZXJpY2EvTG9zX0FuZ2VsZXMifSwiYXNzb3J0bWVudCI6eyJub2RlSWQiOiIzMDgxIiwiZGlzcGxheU5hbWUiOiJTYWNyYW1lbnRvIFN1cGVyY2VudGVyIiwiaW50ZW50IjoiUElDS1VQIn0sImluc3RvcmUiOmZhbHNlLCJkZWxpdmVyeSI6eyJidUlkIjoiMCIsIm5vZGVJZCI6IjMwODEiLCJkaXNwbGF5TmFtZSI6IlNhY3JhbWVudG8gU3VwZXJjZW50ZXIiLCJub2RlVHlwZSI6IlNUT1JFIiwiYWRkcmVzcyI6eyJwb3N0YWxDb2RlIjoiOTU4MjkiLCJhZGRyZXNzTGluZTEiOiI4OTE1IEdlcmJlciBSb2FkIiwiY2l0eSI6IlNhY3JhbWVudG8iLCJzdGF0ZSI6IkNBIiwiY291bnRyeSI6IlVTIiwicG9zdGFsQ29kZTkiOiI5NTgyOS0wMDAwIn0sImdlb1BvaW50Ijp7ImxhdGl0dWRlIjozOC40ODI2NzcsImxvbmdpdHVkZSI6LTEyMS4zNjkwMjZ9LCJpc0dsYXNzRW5hYmxlZCI6dHJ1ZSwic2NoZWR1bGVkRW5hYmxlZCI6dHJ1ZSwidW5TY2hlZHVsZWRFbmFibGVkIjp0cnVlLCJhY2Nlc3NQb2ludHMiOlt7ImFjY2Vzc1R5cGUiOiJERUxJVkVSWV9BRERSRVNTIn1dLCJodWJOb2RlSWQiOiIzMDgxIiwiaXNFeHByZXNzRGVsaXZlcnlPbmx5IjpmYWxzZSwic3VwcG9ydGVkQWNjZXNzVHlwZXMiOlsiREVMSVZFUllfQUREUkVTUyJdLCJzZWxlY3Rpb25UeXBlIjoiREVGQVVMVEVEIn0sInJlZnJlc2hBdCI6MTcwMDU0NTYxMjY4MiwidmFsaWRhdGVLZXkiOiJwcm9kOnYyOmU3NDM5MThmLTljMDEtNDE4NS05ODg5LTAxYjM4M2YzOWE0NiJ9; _tap-googdsp=1700524013260:1700524013261:1; bm_mi=6D08B7F70C2629CB7335921BD0A476C4~YAAQVkDfbybwhNCLAQAAiz8h7xVv+k/UWG2WIDe/gPQ8AOZTnjAhiIOZ0bLj8CiGeNePuQWuPNwhDXS1oNp15tCC1PJSEhUXHiX+NSzkvUL7leDWiBNTKpbglVNmWH1NuG/LvNHQA2LBVlOMNtN+pa4w85WJ5WffbcN5bm4oXQMtRoZmDiaP9LejTQc7RtRDzb/UBiKxSj7B7tWFZWvoaMl7Byh7Zw98qmlF62SYLz85rGSvABr10DN471JChbH4Q/G82+KTkYcWus+IMdzcowTy7Z+Q8iX5zSmhRA2ELuD5Ie109BCzItjTb03IbEwxaZaoK0uGP1lkf74XjjcvevsgHZqUduo=~1; _tap-wmt-dw=1700522135009:0:2; _px3=5ea3ebae35f07f1831578749c875dce471cd3c238e9b1d48cc4d8ac7ed6ef5be:/ONT+vAVQYHGOAb5jG51c6bmUcQjbaW1zryLfFf3ZEFrqWRTeciC0Y4hk/ovL1XeinngRvREckraA7RQaqwaeA==:1000:tUHSmj1FtO2NPUlmIukbTE9SnWpD2T21PLjZBMihCAM89elMZOLre7HSRlFqK71v0f8yL9k4n8Sg8C5OWCQ9mrtXRYNZ7J2cHZIgW/Xriz9FwISvvFB2qUm7ermjWRrawUo7rSOqC6UHNjyG4cNmWupEtwDC+dn8hy+VsaMiY64jQxqICji2ZH+yxxSM8rRGtusit4qn5KaOrVpNR2530EK/u3KBOrrKvimdzwbEfh4=; AID=wmlspartner%3D0%3Areflectorid%3D0000000000000000000000%3Alastupd%3D1700524027817; xptwj=qq:856f56eac45d86299598:ZRnl9M8BUfAYvTYtX+QCNLhEzqw6K+xHmpEjeoxuN7UABA+EM4HcK1WV764ZOO5/6syK6HJNzQ+7oCA1dX0quNAOWtTaQIDcRQupz54KdhbyjvwwBjDPRRo9S45zB814KqqVnZB4xW5GOyMmpEBETfwct40JrJ9yQKgEdWfokI/A+gc=; com.wm.reflector="reflectorid:0000000000000000000000@lastupd:1700524028000@firstcreate:1700292842259"; xptwg=3735093285:1780D552F836C50:3B5C8D6:477BA478:6AAF8424:7C4EFE87:; TS01a90220=01419f1d62bb3082af49e4e290c9e4f7b7d4d09f7a4d1f20d21c60c4e1a7da9ee45eaa7b433f2f9eae578eda14efecc83c7473e1aa; bm_sv=FBAB233769AD51E9AA3317FB35DBDC6D~YAAQVkDfb73yhNCLAQAA5XMh7xUG5PTSMRUK4v2TvEXPwemnWtVYlvkgiCKbkuxNXqf+vVZB8rZcQemv+ydTIl18pO1qMpban0Uf+/ci+ZkYfinA4Fd+fcZ1uHAyQxRGF/plo2v0Gq9TxZgp4pp/YFFuxSnGTqGRKl4RkFJbdxQSVM0CBeOhtUQWXZ0y9L4TE3TF7jw/A9Nu++2UrVU5zLqdE22Fo4Pdw6Yn6RnKbC9OZ7gJsF2DhCgy6+IkYxmg/SE=~1; _pxde=8e9d44f04024aa1db6ed77fcc7bbf79de1d603492bbaba0843e90b5cd14038cf:eyJ0aW1lc3RhbXAiOjE3MDA1MjQwMjg5MTR9',
    'ACID=7cff0725-9085-4be0-bf3c-6839f8621f69; hasACID=true; _m=9; locGuestData=eyJpbnRlbnQiOiJTSElQUElORyIsImlzRXhwbGljaXQiOmZhbHNlLCJzdG9yZUludGVudCI6IlBJQ0tVUCIsIm1lcmdlRmxhZyI6ZmFsc2UsImlzRGVmYXVsdGVkIjp0cnVlLCJwaWNrdXAiOnsibm9kZUlkIjoiMzA4MSIsInRpbWVzdGFtcCI6MTcwMjYzMDM3NDkxNiwic2VsZWN0aW9uVHlwZSI6IkRFRkFVTFRFRCJ9LCJzaGlwcGluZ0FkZHJlc3MiOnsidGltZXN0YW1wIjoxNzAyNjMwMzc0OTE2LCJ0eXBlIjoicGFydGlhbC1sb2NhdGlvbiIsImdpZnRBZGRyZXNzIjpmYWxzZSwicG9zdGFsQ29kZSI6Ijk1ODI5IiwiY2l0eSI6IlNhY3JhbWVudG8iLCJzdGF0ZSI6IkNBIiwiZGVsaXZlcnlTdG9yZUxpc3QiOlt7Im5vZGVJZCI6IjMwODEiLCJ0eXBlIjoiREVMSVZFUlkiLCJ0aW1lc3RhbXAiOjE3MDI2MzAzNzQ5MTUsInNlbGVjdGlvblR5cGUiOiJERUZBVUxURUQiLCJzZWxlY3Rpb25Tb3VyY2UiOm51bGx9XX0sInBvc3RhbENvZGUiOnsidGltZXN0YW1wIjoxNzAyNjMwMzc0OTE2LCJiYXNlIjoiOTU4MjkifSwibXAiOltdLCJ2YWxpZGF0ZUtleSI6InByb2Q6djI6N2NmZjA3MjUtOTA4NS00YmUwLWJmM2MtNjgzOWY4NjIxZjY5In0%3D; vtc=c_lPqL_NYDC6UXPeh9MF7U; _pxvid=5644c6ff-9b27-11ee-ac94-b6ae3dc3013d; pxcts=578a7ded-9b27-11ee-b1db-83fbf11618da; thx_guid=f069aa2ed76a293b146f2f45273ffc5e; QuantumMetricUserID=86dbb1aaa4c0846b6472227c8127c115; bm_mi=A9B761149346E34C2336993088CBA838~YAAQpCLHF0tB41iMAQAAT6lrexYSWryWW33ix0ebW0gEwR9R9Q3T0mju/pC7V7LqjuwHJrGseRkgFSZZiZVHV/zjTu4nCYLngf6698jYoiODJd20Ah32C8vCA7fNXyHUhtfSgpdKNkkOpAYXO8oYiiVKF1l5IHZeILy9fj41t1W9tF9DODYuxTNU8QPPnZr4w70z23lcK+3EBKd8IfqBgPT9kjLT3pKPY5P1EjKBk7Wvn9d44E3YtdQOdIlnmZMFqcniDHXEd0TgXXrvQ7NVo1BkEMaGynnbyeApl64EsIYaBB5nCsrN2NFOY0C1ZumHLPsxg5pg~1; ak_bmsc=28DD2AB37291D5EE54D348A14FFB7027~000000000000000000000000000000~YAAQpCLHF1FB41iMAQAA3atrexY3RQO5uSrgAn4o71OTeCY3Kb4oklPOhzO4ob+ZppkPcKbMuYwUEtBs6vEVlp9CoeIcig2wNR+YFPAMLag0FgMkBgPTNS7DAEU7mXMzqG7Zxes7pw+1iCYIJKVm0KAso3OE50SIiVGnpFKhhL7E2/cDwpWbUdrNVY30TGqqMk/uCOYcvkqNQE0wzVpTYkK2hWIeztHPT5fNqkdtRUylfN9FBTzfaUxix+emgkOw4S/zZxgkw4lNSo/YbiRZDpT409NuUxV4ht5j47adbQ1Q8F08pAvMkK3vGzOypF53bdu9Iw3g3D4PUZDOdQ2FFArxdUalCDwieOVrUbY5Xu55+30+fnRIFgzDM+aa72JMrHteO5326b1oz/QcUX1+EgWPmeSXJTk6/0EJvYS6yU9fnzsad5XVkCe4cRcS0AP/ZVfbCDorHjsVEachEwRjpKmMWx8q6DOuGFw00I1vRusHKdW1nrCknd+TjPY2u042+mVSAQCo0JoYGB8VZlNhvUof6KzcVkfB4ZG5; auth=MTAyOTYyMDE45uQqkShnPmrFyDklTStmr7wc0IXA0uhqKhIjidzMeOlor0FoZrStjvQEbiRm5ZqSmQEJ%2FPunCsfySE6LPKFrLFOCiFQAqcgu3n0iVe3t%2BsYE2L20v13oIDsvWtwF1M8S767wuZloTfhm7Wk2Kcjygi5k0VvBM%2FJjwcKWWhCnBS%2FsNVBmy9J1bR2VHO%2FdV8LIpQmp8XOq309QoW%2BZviaSOv8sqUVU54sFd4Bd6dus2ZEUMk70P8glgOEpLOprhDfMDCcb9mgycy9jtT1uIyOBHVeQqO76rPmdncwFjpe%2BDY%2BOi7voHCqN3McdaIPwoybp22ykAEJaZ9rj2LqLTXsr9v3%2FEocn3Z%2BtLU09JmG1TrtCvNQVjLp%2Ffv%2BF%2BvOqp4HL4oel5ASvElr1Cex8QK9UZEjyrOXbKKhH072NS%2FW0j%2FU%3D; bstc=VDdH8jJ4m0c1jwDTa0PzoE; mobileweb=0; xptc=assortmentStoreId%2B3081; xpth=x-o-mart%2BB2C~x-o-mverified%2Bfalse; xpa=1mX_Y|7C2Eg|ERspM|HhvdQ|IS-p_|LPh6f|XbHrX|csP8O|gUDh7|oY0CV|qKfBf|yamTG; exp-ck=IS-p_1LPh6f1XbHrX1gUDh71qKfBf1; xpm=1%2B1702877712%2Bc_lPqL_NYDC6UXPeh9MF7U~%2B0; QuantumMetricSessionID=fb9f81cd396976e7c5f3318d82cf9aca; AID=wmlspartner%3D0%3Areflectorid%3D0000000000000000000000%3Alastupd%3D1702878308583; xptwj=qq:1076adbafd040b695fce:7QGLRUcehov3qVy4qSYDvW4xgk6Uac2yX7HX1tE4u646HYe83hDIeMje3Thc/1gvh8z8MrbJNvYdlJL4OmUTikr/fII5bRfC3yxM6l2D8uZIkuA2LrEs6TpjeeKaJ8Uu78B/EOBkj1BzXZ7N6125enlMLSzG; xptwg=3146300320:19444130883AE60:3FBA1BE:1196AC61:D2F58FD2:22F02742:; _px3=6c9349297a823e7545b893fcfa8e6bea6eecb57dc1b11b7cf64522744342fc05:q5wbXoTzZgQ7QP9YgqzSywMmiYqoqw/ZrncImDY3CrTPeEkOqVzPo2zJ3AHIc+s/Ltf7IlS8awVgjpNXM98pxQ==:1000:cn/ohrnROd+TGJA88Xeo4/KYIo/UsLvu6vSoOqMnmEug9QezGcx/WE0rfSuHGKZjjjaYrqXq8pBrJ5nlgEeADLsTsoKQiFm1rc/m/6Q+wMMNleiOZAl9VZy3wpYUymfUCrFwGU+qqgFlXPCfQNN9o6vjljQksDltmcFQIof95UjvNs7UMihymHVA7zlP5BOXCURkIHiiJbM73dOLjmbFMFnxGHKXULDB8u2cb1oECR0=; com.wm.reflector="reflectorid:0000000000000000000000@lastupd:1702878315000@firstcreate:1702630374882"; TS01a90220=014e9abc5b76ea37289d24aa7bf6872327f3a801259b614de94e2e0e90c1b0f94c88be0edeb3c58cf3cba5b3568b2b79f66cc4dc9c; bm_sv=ADF605AFD1AA1E340A3C57CE2E012D0E~YAAQJPN0aPSze2mMAQAAbQJ1exaqQAmPQEugdiG3MQAUrup+aS98eOHJhIKWYEY4cQ16kvxr5PjD68+TG/q08uWKgkntuhl4BsnOGqcEXY7GjWBoB0neW+58ysLmfhee1rCKsVjS3iaBCoFZwcqIgkO3eDhiRFyKZXfNTkKDX7JBTyy63S1DAg2SrohZrD0KlpE5E2rFPlHxbpWohjTWhvXGlDIHA0lk8uQB+NvqN5R8yDL6bMxSm0YrJ6/p6h5kxvQ=~1; _pxde=3cb689d88ebd75dfd5e5c64a8389623aecf6bac16e69f216cf01a4e825c18d52:eyJ0aW1lc3RhbXAiOjE3MDI4NzgzMTUzNDl9',
    'ACID=6ff15283-acba-4fe2-89c5-ac59a9b887d1; hasACID=true; thx_guid=0c376c1283b8b14be6de36c7c9897b80; AID=wmlspartner%3D0%3Areflectorid%3D0000000000000000000000%3Alastupd%3D1696941267921; _m=9; vtc=WYxqCClwKT085jYR_DbfQI; _pxvid=5a44b304-6769-11ee-a5a3-89fdf7f6e6ba; auth=MTAyOTYyMDE4IIJ07VmsLMduZmu3wWCW8eIqM67RZKVCKQnP%2BJhyprb8mcLFoQ86xyheUk7V1wBI53SlNalpIeiZEAnwibnwBxAuKnHyan446S83cruRp5IqJEEhjLzfZ9dTkdF%2F5mvs767wuZloTfhm7Wk2Kcjygt6CFmh5hT8BoAhiLFQG8TM4tK7YyL%2Bjr93Ekvm3gtoWXd1A1TJkrpfzbS%2B%2BXZ2ssMBCHcdxxw3SP0Sy3y18bhsUMk70P8glgOEpLOprhDfMDCcb9mgycy9jtT1uIyOBHfClvOkjwxW8L0euuWDrN9AOTwJZ5k6XdH2IzYdedb%2BeKcBnww%2BqCeKjSX3bV3tRPjMVnRfkQSZ38Y3kHRhf5YWMEw3bsV%2BTWUdfiZ61nY1rm2KT4Gr0iVCCeIJhV8GhuUjyrOXbKKhH072NS%2FW0j%2FU%3D; locDataV3=eyJpc0RlZmF1bHRlZCI6dHJ1ZSwiaXNFeHBsaWNpdCI6ZmFsc2UsImludGVudCI6IlNISVBQSU5HIiwicGlja3VwIjpbeyJidUlkIjoiMCIsIm5vZGVJZCI6IjMwODEiLCJkaXNwbGF5TmFtZSI6IlNhY3JhbWVudG8gU3VwZXJjZW50ZXIiLCJub2RlVHlwZSI6IlNUT1JFIiwiYWRkcmVzcyI6eyJwb3N0YWxDb2RlIjoiOTU4MjkiLCJhZGRyZXNzTGluZTEiOiI4OTE1IEdlcmJlciBSb2FkIiwiY2l0eSI6IlNhY3JhbWVudG8iLCJzdGF0ZSI6IkNBIiwiY291bnRyeSI6IlVTIiwicG9zdGFsQ29kZTkiOiI5NTgyOS0wMDAwIn0sImdlb1BvaW50Ijp7ImxhdGl0dWRlIjozOC40ODI2NzcsImxvbmdpdHVkZSI6LTEyMS4zNjkwMjZ9LCJpc0dsYXNzRW5hYmxlZCI6dHJ1ZSwic2NoZWR1bGVkRW5hYmxlZCI6dHJ1ZSwidW5TY2hlZHVsZWRFbmFibGVkIjp0cnVlLCJodWJOb2RlSWQiOiIzMDgxIiwic3RvcmVIcnMiOiIwNjowMC0yMzowMCIsInN1cHBvcnRlZEFjY2Vzc1R5cGVzIjpbIlBJQ0tVUF9JTlNUT1JFIiwiUElDS1VQX0NVUkJTSURFIl0sInNlbGVjdGlvblR5cGUiOiJMU19TRUxFQ1RFRCJ9XSwic2hpcHBpbmdBZGRyZXNzIjp7ImxhdGl0dWRlIjozOC40NzQ1LCJsb25naXR1ZGUiOi0xMjEuMzQzOCwicG9zdGFsQ29kZSI6Ijk1ODI5IiwiY2l0eSI6IlNhY3JhbWVudG8iLCJzdGF0ZSI6IkNBIiwiY291bnRyeUNvZGUiOiJVU0EiLCJnaWZ0QWRkcmVzcyI6ZmFsc2UsInRpbWVab25lIjoiQW1lcmljYS9Mb3NfQW5nZWxlcyJ9LCJhc3NvcnRtZW50Ijp7Im5vZGVJZCI6IjMwODEiLCJkaXNwbGF5TmFtZSI6IlNhY3JhbWVudG8gU3VwZXJjZW50ZXIiLCJpbnRlbnQiOiJQSUNLVVAifSwiaW5zdG9yZSI6ZmFsc2UsImRlbGl2ZXJ5Ijp7ImJ1SWQiOiIwIiwibm9kZUlkIjoiMzA4MSIsImRpc3BsYXlOYW1lIjoiU2FjcmFtZW50byBTdXBlcmNlbnRlciIsIm5vZGVUeXBlIjoiU1RPUkUiLCJhZGRyZXNzIjp7InBvc3RhbENvZGUiOiI5NTgyOSIsImFkZHJlc3NMaW5lMSI6Ijg5MTUgR2VyYmVyIFJvYWQiLCJjaXR5IjoiU2FjcmFtZW50byIsInN0YXRlIjoiQ0EiLCJjb3VudHJ5IjoiVVMiLCJwb3N0YWxDb2RlOSI6Ijk1ODI5LTAwMDAifSwiZ2VvUG9pbnQiOnsibGF0aXR1ZGUiOjM4LjQ4MjY3NywibG9uZ2l0dWRlIjotMTIxLjM2OTAyNn0sImlzR2xhc3NFbmFibGVkIjp0cnVlLCJzY2hlZHVsZWRFbmFibGVkIjp0cnVlLCJ1blNjaGVkdWxlZEVuYWJsZWQiOnRydWUsImFjY2Vzc1BvaW50cyI6W3siYWNjZXNzVHlwZSI6IkRFTElWRVJZX0FERFJFU1MifV0sImh1Yk5vZGVJZCI6IjMwODEiLCJpc0V4cHJlc3NEZWxpdmVyeU9ubHkiOmZhbHNlLCJzdXBwb3J0ZWRBY2Nlc3NUeXBlcyI6WyJERUxJVkVSWV9BRERSRVNTIl0sInNlbGVjdGlvblR5cGUiOiJMU19TRUxFQ1RFRCJ9LCJyZWZyZXNoQXQiOjE2OTg0MDA0ODkwNzgsInZhbGlkYXRlS2V5IjoicHJvZDp2Mjo2ZmYxNTI4My1hY2JhLTRmZTItODljNS1hYzU5YTliODg3ZDEifQ%3D%3D; locGuestData=eyJpbnRlbnQiOiJTSElQUElORyIsImlzRXhwbGljaXQiOmZhbHNlLCJzdG9yZUludGVudCI6IlBJQ0tVUCIsIm1lcmdlRmxhZyI6ZmFsc2UsImlzRGVmYXVsdGVkIjp0cnVlLCJwaWNrdXAiOnsibm9kZUlkIjoiMzA4MSIsInRpbWVzdGFtcCI6MTY4NTc4NzY5OTA3OSwic2VsZWN0aW9uVHlwZSI6IkxTX1NFTEVDVEVEIn0sInNoaXBwaW5nQWRkcmVzcyI6eyJ0aW1lc3RhbXAiOjE2ODU3ODc2OTkwNzksInR5cGUiOiJwYXJ0aWFsLWxvY2F0aW9uIiwiZ2lmdEFkZHJlc3MiOmZhbHNlLCJwb3N0YWxDb2RlIjoiOTU4MjkiLCJjaXR5IjoiU2FjcmFtZW50byIsInN0YXRlIjoiQ0EiLCJkZWxpdmVyeVN0b3JlTGlzdCI6W3sibm9kZUlkIjoiMzA4MSIsInR5cGUiOiJERUxJVkVSWSIsInRpbWVzdGFtcCI6MTY5ODM5Njg4OTA3Miwic2VsZWN0aW9uVHlwZSI6IkxTX1NFTEVDVEVEIiwic2VsZWN0aW9uU291cmNlIjpudWxsfV19LCJwb3N0YWxDb2RlIjp7InRpbWVzdGFtcCI6MTY4NTc4NzY5OTA3OSwiYmFzZSI6Ijk1ODI5In0sIm1wIjpbXSwidmFsaWRhdGVLZXkiOiJwcm9kOnYyOjZmZjE1MjgzLWFjYmEtNGZlMi04OWM1LWFjNTlhOWI4ODdkMSJ9; bstc=RDjQg_AX5lxPJuFgq3Nu74; mobileweb=0; xpth=x-o-mart%2BB2C~x-o-mverified%2Bfalse; xpa=92YHy|YnYws|yUqGy; exp-ck=YnYws4; pxcts=7ca9260b-74a6-11ee-b9ba-cadc77ff3193; xptc=assortmentStoreId%2B3081; xpm=1%2B1698396890%2BWYxqCClwKT085jYR_DbfQI~%2B0; TS01a90220=016ea84bd28377d28f6c5f8c825a73acff43907723c0c62c3b0f4412526365096a8d6e578fadcc7b7c5313bbb50b7e1d6a09790715; xptwj=qq:8c955083c3f2d971e73b:/AUbzj+G4qHiFzoBmTqw7sTxNCjaAsdlFcEBElDEMhfux6fBocZ2XuU9J43MZqv9xFdQR6jCZn1NHMlGARop+dGCrSwqG/FMfkjiqrzMhk6qDRXnGwdXkmVMkY17cQBnEuy2yF31oDbLB2VcFQm/6GzBoQa2xx4u4jyJ; com.wm.reflector="reflectorid:0000000000000000000000@lastupd:1698397083000@firstcreate:1696941267921"; xptwg=2654488666:13D200536871A40:321F50E:2AE8C8EB:145FE951:E3F58838:; _px3=2259e2ff82c009ea11f3276f67eeec8c7933f2015f7c4f4f537a810e135667b2:WTyoBm43yjHnl6DaKRhXrBuw1dVxFeMmNao4s1nyzyzay9AT/JNRr2njauNA3Q05CZCkyHcGpiOlACvkOPaeLA==:1000:eqWykI7N/3Nxz6qb3xQH+stpzInFVztcX104+VKDHUoglkWCA2mLjEu+Zknx3FqHY6MZskTdWOhU7b/cxkcRsZzn2v7xd2d0SIFOVaBuJFFm4ddlo4ejUXkO/Ta7SH2GcvS0zq5pIVgCSlg9SjcmMala24rEipeabfBpgSjY1DFP4u3vQ5vVD9nlh6dHTGDAN6J86YkunMWOWKq/mltB5LLWDt5U+hsHxwlUQFOl53E=; _pxde=67376920231ea198f43cb2a2fd4e3570e6f0c4527d7e9406b6848b481178e72e:eyJ0aW1lc3RhbXAiOjE2OTgzOTcwODU3NDh9',
    'ACID=6ff15283-acba-4fe2-89c5-ac59a9b887d1; hasACID=true; AID=wmlspartner%3D0%3Areflectorid%3D0000000000000000000000%3Alastupd%3D1696941267921; _m=9; locGuestData=eyJpbnRlbnQiOiJTSElQUElORyIsImlzRXhwbGljaXQiOmZhbHNlLCJzdG9yZUludGVudCI6IlBJQ0tVUCIsIm1lcmdlRmxhZyI6ZmFsc2UsImlzRGVmYXVsdGVkIjp0cnVlLCJwaWNrdXAiOnsibm9kZUlkIjoiMzA4MSIsInRpbWVzdGFtcCI6MTY4NTc4NzY5OTA3OSwic2VsZWN0aW9uVHlwZSI6IkxTX1NFTEVDVEVEIn0sInNoaXBwaW5nQWRkcmVzcyI6eyJ0aW1lc3RhbXAiOjE2ODU3ODc2OTkwNzksInR5cGUiOiJwYXJ0aWFsLWxvY2F0aW9uIiwiZ2lmdEFkZHJlc3MiOmZhbHNlLCJwb3N0YWxDb2RlIjoiOTU4MjkiLCJjaXR5IjoiU2FjcmFtZW50byIsInN0YXRlIjoiQ0EiLCJkZWxpdmVyeVN0b3JlTGlzdCI6W3sibm9kZUlkIjoiMzA4MSIsInR5cGUiOiJERUxJVkVSWSIsInRpbWVzdGFtcCI6MTY5Njk0MTI2ODAwMSwic2VsZWN0aW9uVHlwZSI6IkxTX1NFTEVDVEVEIiwic2VsZWN0aW9uU291cmNlIjpudWxsfV19LCJwb3N0YWxDb2RlIjp7InRpbWVzdGFtcCI6MTY4NTc4NzY5OTA3OSwiYmFzZSI6Ijk1ODI5In0sIm1wIjpbXSwidmFsaWRhdGVLZXkiOiJwcm9kOnYyOjZmZjE1MjgzLWFjYmEtNGZlMi04OWM1LWFjNTlhOWI4ODdkMSJ9; userAppVersion=us-web-1.102.0-0f3d752097f13fd03499487f7cfc0f9ff879d809-1005; abqme=true; vtc=WYxqCClwKT085jYR_DbfQI; _pxhd=5a7ffd639284c9b62b5b6953d2b6554b5e4fb23e72bdea13cb0d60c5e9cb2592:5a44b304-6769-11ee-a5a3-89fdf7f6e6ba; TBV=7; _pxvid=5a44b304-6769-11ee-a5a3-89fdf7f6e6ba; pxcts=5b1312c1-6769-11ee-9d4b-928400606778; xptwj=qq:19ae55e85ed74ecb934a:FzwixoJNbjsJTKIVOxs2Y3BCAjYnbpEJ9QAEPF+vcgu7rou9eHViyjDPVj+jQqEQsDVe8eLUcM9yr4bzIXF5/EpE+3GBy+nQfjIux03VKMmH4uP0zvUVBAnki5gXoud346PderEXI4ZdwzI5dEw9RZpxrSE=; _astc=dd455cd93be2a8805fa78a0c5637c0bc; com.wm.reflector="reflectorid:0000000000000000000000@lastupd:1696943665000@firstcreate:1696941267921"; xptwg=3827596973:8C0849C3A8C7E8:1626C14:C8D91831:9FD2BC9F:ED33D50A:; TS012768cf=0178545c900bf5c440f69b21bbdea7b97f1bb93829c83cdf12d8a829eaa1f335330ed161b9a95404539366655b7d1af2acfeaa823d; TS01a90220=0178545c900bf5c440f69b21bbdea7b97f1bb93829c83cdf12d8a829eaa1f335330ed161b9a95404539366655b7d1af2acfeaa823d; TS2a5e0c5c027=0881c5dd0aab20006278b8f1c282bae24adaf3370d4de18765bc5b96e8045bf269a5218310a2b5f308691a3aad113000c4a62f799fcd6ee383055c96bd53eb55adecf63f1102cc5dd537a8a8e81ee1756147c51df92a6c584fb9a07c447c0309', 
    'ACID=13f858d3-9165-43cb-bab4-a63c55e6a6a8; hasACID=true; _m=9; locGuestData=eyJpbnRlbnQiOiJTSElQUElORyIsImlzRXhwbGljaXQiOmZhbHNlLCJzdG9yZUludGVudCI6IlBJQ0tVUCIsIm1lcmdlRmxhZyI6ZmFsc2UsImlzRGVmYXVsdGVkIjp0cnVlLCJwaWNrdXAiOnsibm9kZUlkIjoiMzA4MSIsInRpbWVzdGFtcCI6MTcwMjk1NTMwNTIxNSwic2VsZWN0aW9uVHlwZSI6IkRFRkFVTFRFRCJ9LCJzaGlwcGluZ0FkZHJlc3MiOnsidGltZXN0YW1wIjoxNzAyOTU1MzA1MjE1LCJ0eXBlIjoicGFydGlhbC1sb2NhdGlvbiIsImdpZnRBZGRyZXNzIjpmYWxzZSwicG9zdGFsQ29kZSI6Ijk1ODI5IiwiY2l0eSI6IlNhY3JhbWVudG8iLCJzdGF0ZSI6IkNBIiwiZGVsaXZlcnlTdG9yZUxpc3QiOlt7Im5vZGVJZCI6IjMwODEiLCJ0eXBlIjoiREVMSVZFUlkiLCJ0aW1lc3RhbXAiOjE3MDI5NTUzMDUyMTQsInNlbGVjdGlvblR5cGUiOiJERUZBVUxURUQiLCJzZWxlY3Rpb25Tb3VyY2UiOm51bGx9XX0sInBvc3RhbENvZGUiOnsidGltZXN0YW1wIjoxNzAyOTU1MzA1MjE1LCJiYXNlIjoiOTU4MjkifSwibXAiOltdLCJ2YWxpZGF0ZUtleSI6InByb2Q6djI6MTNmODU4ZDMtOTE2NS00M2NiLWJhYjQtYTYzYzU1ZTZhNmE4In0%3D; vtc=bqXm-Yjh0fiyfC30O9pl40; pxcts=e0a91fc6-9e1b-11ee-b6a5-16e7f3c0c35f; _pxvid=dfcedab4-9e1b-11ee-852d-cc6ba9195b8b; thx_guid=d45cea97b66ab1e12333b05cf300756a; auth=MTAyOTYyMDE4eh0UBw4CZsMoITCUpLy%2FJSKZAhO1G27GRMvlySSJu45p1%2FLh4CYkzkJMDMayVQPkr4cn%2Flzfu3Cnm2UosjL24mxfJqc48PMpEXdaud0aTiSYybEajj235v6w6v39wTDg767wuZloTfhm7Wk2KcjygobRHThsmZk%2BGcqTfIab85Qi91RLvjJ4oWxX7pdsgCM7kaNdhw7fWS2J7XYV98BtLp94fDX6wtiILXdT4QaPibQUMk70P8glgOEpLOprhDfMDCcb9mgycy9jtT1uIyOBHYAmxWm2QCSM81oB%2BzgtGh7GRphnRVqhmKz4T4aeRpfPdIrk6V7SOwO2Q2sHD6RhS27h8BprVsmSYkJBi2ZANdkS%2BmkqvUibwJ%2ByNBdR4lDAXTGRW5wSkfxBkI28si3Kp0jyrOXbKKhH072NS%2FW0j%2FU%3D; bstc=eiIdhbpg62SIN7bXqMg3z8; mobileweb=0; xptc=assortmentStoreId%2B3081; xpth=x-o-mart%2BB2C~x-o-mverified%2Bfalse; xpa=1mX_Y|M65NP|WuHSe|XmyU7|j21L4|oY0CV|qKfBf|qMQpD|r97uO; exp-ck=WuHSe1XmyU71j21L41qKfBf1r97uO1; xpm=1%2B1702962355%2BbqXm-Yjh0fiyfC30O9pl40~%2B0; bm_mi=549FCFD96C94678551FEF51A3881B212~YAAQX/N0aH0UHFmMAQAAD7F9gBYxuYoIp+e5g1nqvEyexI6hSo7fx80xYZvYxOzX42gMoKUiODVyjrfCTZxzUPZoY9b4p074yFBo9qtr3dvqcrWK+Ax3QnilXHYN/O3i9uGgMgaYzDbvFsofcB+UXepCydTAGXEwlgTjcuhCK/WHSSkwgTYZBCgoH4ZvNuWxbDqSZJVHpneNw6xdnWnxgDwCiwgTl9vrLyjdg09e4T2smzNssFE245sbVMdbiBsLplNv/n7syfFu/VVXuZyCtMvNaORC85FiXoDfVzz0LARwtlqQLr1WRjGOEklBkKaf65Mxcoy8~1; ak_bmsc=2EB05928C59E3D09719A985EA17D031E~000000000000000000000000000000~YAAQX/N0aIcUHFmMAQAAg7N9gBbfI4TKoREfgSIqsmk7VQxNHd1hZRptwqQOz86rLpb4Dm8hwXmGCtpOkNohVRQHY/oG6ctA+y0cssVoASKpTSd1fEYyHDqbC+DrYiN6LJ0q930S+F4z3j6ktqalEJm6Bh4BtsGU6d5h43XeXgGRDzTLgZf+D5taWBpWzCUAb/xXmehSq77svJniJ4HmGoQYRn4fH8UeuB958xg08dR7TF3qVNpEUpqlv1WV4x9fcXDK35s9YBcHcZJUpGXMIx0+3rA/JVyuVqSLJnTzu9yOhVYgISi07np3590KzU5WW1U9oTYdQVYyvkMD3EL1LbLzXiZ10P9fl+HnFHX0V52zIFkboWpvSkdK4HVlAR/7LrWEM6PaexEzR6dCESt+Az+opps+z9JZNT6Y9W2aalGBPdzj275+3AznMTCf3hNispVbb4Im9fKO4HjsyY/rQKVgXwkJ6+IVYPDvYRFJaS1jY42zuoQknNidJ0ajbhHEwI5WkMGbsQ6ZfxAYYFUHPOj4ZWUDO1N9gnSx; AID=wmlspartner%3D0%3Areflectorid%3D0000000000000000000000%3Alastupd%3D1702963573470; xptwj=qq:60c655ed12ee68048e72:54KBHIYpe8ioxt3yuo3GuRpC1WCSieQoP/HSZ4zw05vUNBqTEGjZylRV7Ee/gQKdw+4AR39Pu3s9w8UN6fC+xQaysEu/16eRafxBSlOhJPWWTVuI1BF3b300k0De/jcPtI82jkh9uGX2O4ufX9jSWMwn87b+BBU=; com.wm.reflector="reflectorid:0000000000000000000000@lastupd:1702963574000@firstcreate:1702955305178"; xptwg=4227679326:558030DE400990:D7A337:D1486F6E:AEABFA5B:A48C033E:; TS01a90220=0178545c90c924ff084263c8cb94d8a93feb10ba5dcdaafa44d2d4bbee53181a33b55ab3c28c48628f7c56388b13afbe855a015aeb; bm_sv=60E1D169192E935CC6C00B0CDE127AEB~YAAQniLHF0lo0UKMAQAAEPaJgBa2H14Xw8z9j5FuLe+c+mOr9vtjWy+B1JIWMPOzFV3jf2sOwlAcIEVT7Zb0mGwx9GetlyN6e/PNbuNPGRK0tdoA/hf8gjCduTwLS8CC6z4no3eEnvY1ag+q3Dq0y4dikxDbYXvcfVujjYrVNac/v5cOfWcW40fMWhkJQX7hGxnZqmIyc+52KM8NeP9eIqEdybm44398ucnnT/mCybgp/Bn9xYLZwQBJSE7edA60x3A=~1; _px3=6e98f47b3f022f412f75d2fff334dd78b42e1c2da9dbb1ac36bcae228b1359cb:Vj9ElCDReT1PHAkl34AbO/7bhYH1iIhL+fexBxQwF/yJALJP177TUZi0QsuawJtQWCRpPtcCQy7FPqF2SO+Gpw==:1000:x43ZNK+0w/plDdsG3NfS1bezpRm0D1++lXrQuSJabg6/xCZEc7Ghp02iHShQZQWtATaAUPAyVYeZC2R8lPF2X7xPgH9OSstdEZ2g17OOrJ4Fv7E1MK6KZq0DiVn7FxyrcRAXcBHekyDMGiK6TTAugoF3QVB4LAQ7WM6kpnEbXlQLvndZW/IDW1JNGgVGCJvdRpV+VpqD6Ab7P2yv3UX4n+cRwoKWIa99iFjoZbmOuYM=; _pxde=3810374111437fdd342c82c0dbe956b1fb4871c6ade129e930a96584572ba38e:eyJ0aW1lc3RhbXAiOjE3MDI5NjM1NzUwNDJ9'
]


In [ ]:
def get_page_number(url, cookie):
    User = random.choice(User_Agent)
    header = {
        'User-Agent': User,
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9',
        'Cache-Control': 'max-age=0',
        'Cookie': random.choice(cookie),  # Replace with the actual Cookie
        'Downlink': '10',
        'Dpr': '1',
        'Referer': url,
        'Sec-Ch-Ua': '"Google Chrome";v="117", "Not;A=Brand";v="8", "Chromium";v="117"',
        'Sec-Ch-Ua-Mobile': '?0',
        'Sec-Ch-Ua-Platform': '"Windows"',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-Fetch-User': '?1',
        'Upgrade-Insecure-Requests': '1'
    }
    response = requests.get(url, headers=header)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')
    page_number_elements = soup.find_all(lambda tag: tag.name == 'a' and 'page-number' in tag.get('data-automation-id', ''))
    page_numbers = [int(element.text) for element in page_number_elements]
    if page_numbers:
            last_page_number = max(page_numbers)
            return last_page_number
    elif "Robot or human" in response.text:
        print(User)
        print('Robot or human page')
    else:
        print(False)
        

def get_review_walmart(url, cookie):
    extracted_reviews = []
    retry_count = 0
    header = {
         'User-Agent': random.choice(User_Agent), 
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9',
        'Cache-Control': 'max-age=0',
        # 'Cookie': random.choice(cookie),
        'Downlink': '10',
        'Dpr': '1',
        'Referer': url,
        'Sec-Ch-Ua': '"Google Chrome";v="117", "Not;A=Brand";v="8", "Chromium";v="117"',
        'Sec-Ch-Ua-Mobile': '?0',
        'Sec-Ch-Ua-Platform': '"Windows"',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-Fetch-User': '?1',
        'Upgrade-Insecure-Requests': '1'
    }

    try:
        response = requests.get(url, headers=header)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        li_elements = soup.find_all('li', class_='dib w-100 mb3')
        title_all = soup.find('a',class_ = 'w_x7ug f6 dark-gray')
        if title_all:
            title = title_all.get('href')
            pattern = r'(\d{4}[a-zA-Z]?)-'
            model = re.findall(pattern, title)
#         model = re.search(r'\b(\d{4}e)\b', title_all).group(1)
        li_elements = soup.find_all('li', class_='dib w-100 mb3')
        
        if li_elements:
            for li_tag in li_elements:
                product = {}  # Create a new 'product' dictionary for each review
                product['Model'] = title
                product['Review rating'] = li_tag.select_one('.w_iUH7').text
                product['Verified Purchase or not'] = li_tag.select_one('.pl2.green.b.f7.self-center').text if li_tag.select_one('.pl2.green.b.f7.self-center') else None
                product['Review date'] = li_tag.select_one('.f7.gray').text if li_tag.select_one('.f7.gray') else None

                review_title_element = li_tag.select_one('h3.b')
                product['Review title'] = review_title_element.text if review_title_element else None

                product['Review Content'] = li_tag.find('span', class_='tl-m mb3 db-m').text if li_tag.find('span', class_='tl-m mb3 db-m') else None
                product['Review name'] = li_tag.select_one('.f6.gray').text if li_tag.select_one('.f6.gray') else None

                syndication_element = li_tag.select_one('.b.ph1.dark.gray')
                product['Syndicated source'] = syndication_element.text if syndication_element else None
                product['URL'] = url

                extracted_reviews.append(product)
        elif "Robot or human" in response.text:
            print('Robot or human page')
            

    except requests.exceptions.RequestException as e:
        print(f"Request error encountered: {e}. Retrying in 5 seconds...")
        time.sleep(5)
    except Exception as e:
        print(f"Error encountered: {e}. Retrying in 5 seconds...")
        time.sleep(5)

    return extracted_reviews

## HP

In [ ]:
urls = [
    'https://www.walmart.com/reviews/product/651507182?sort=submission-desc',
    'https://www.walmart.com/reviews/product/716614887?sort=submission-desc'
] 



In [ ]:
import time

walmart_reviews = []
# urls = [
#     'https://www.walmart.com/reviews/product/651507182?sort=submission-desc',
#     'https://www.walmart.com/reviews/product/716614887?sort=submission-desc'
# ]
for link in urls:
    retry_count = 0
    max_try = 20
    print(link)
    while True:
        try:
            last_page_number = get_page_number(link, cookie)
            if last_page_number is None:
                print("Failed to retrieve last page number. retry...")
                time.sleep(3)
                continue  
            print('Total pages:', last_page_number)
            break 
        except Exception as e:
            print(f"Error encountered: {e}. Retrying in 3 seconds...")
            time.sleep(3)


    for page_number in range(1, last_page_number + 1):
        while retry_count < max_try:
            try:
                target_url = f'{link}?page={page_number}'
                extracted_reviews = get_review_walmart(target_url, cookie)

                if len(extracted_reviews) == 0:
                    print('No reviews found. Retrying in 5 seconds...')
                    retry_count += 1
                    time.sleep(5)
                else:
                    walmart_reviews.extend(extracted_reviews)
                    print(f'Review count in page {page_number}:', len(extracted_reviews))
                    time.sleep(2)
                    break  

            except Exception as e:
                print(f"Error encountered: {e}. Retrying in 3 seconds...")
                retry_count += 1
                time.sleep(3)
        else:
            break 
    else:
        continue 
    
    break


In [ ]:
path = r"C:\Users\TaYu430\anaconda3\envs\webscrap\My Scripts\Star rating scrape URL and info - NPI.xlsx"
sheets = 'data_new'
df_walmart = pd.read_excel(path, sheet_name = sheets)
df_walmart['Comp Model number'] =  df_walmart['Comp Model number'].astype(str)
df_walmart['HP Model Number'] =  df_walmart['HP Model Number'].astype(str)
df_walmart

In [ ]:
walmart= pd.DataFrame(walmart_reviews)
walmart['Retailer']="walmart"

from datetime import date 
walmart['scraping_date'] = date.today().strftime('%Y/%m/%d')
walmart['scraping_date'] = pd.to_datetime(walmart['scraping_date']).dt.date
walmart['Review date'] = pd.to_datetime(walmart['Review date']).dt.date
walmart['Review rating'] = walmart['Review rating'].astype(str).str.replace(' out of 5 stars review', '').astype(int)
walmart.drop_duplicates(inplace = True)

walmart['HP Model Number'] = walmart['Model'].str.extract(r'(\d+e?)')

walmart['Review date'] = pd.to_datetime(walmart['Review date'])

walmart_hp_combine = pd.merge(walmart, df_walmart, on = "HP Model Number", how = "left" )
walmart_hp_combine['Review Model'] = walmart_hp_combine['HP Model'] 
columns_to_drop = [  
    'Model', 'HP Model Number', 'Comp Model number','HP Model'
]  
  
walmart_hp_combine = walmart_hp_combine.drop(columns_to_drop, axis = 1) 

walmart_hp_combine = walmart_hp_combine.drop_duplicates()
walmart_hp_combine['Competitor_Flag'] = walmart_hp_combine['Review Model'].apply(lambda x: 'No' if 'HP' in x else 'Yes')
walmart_hp_combine['Country'] = 'US'

column_mapping = {
    'Review date': 'Review_Date',
    'review_text': 'Review_Content',
    'Review rating':'Review_Rating',
    'url': 'URL',
    'review_title': 'Review_Title',
    'Verified Purchase or not': 'Verified_Purchase_Flag',
    'reviewer_name': 'Review_Name',
    'syndication': 'Syndicated_Source',
    'stars': 'Review_Rating',
    'Retailer': 'Retailer',
    'scraping_date': 'Scraping_Date',
    'Comp Model': 'Comp_Model',
    'HP Class': 'HP_Class',
    'Review Model': 'Review_Model',
    'Review title':'Review_Title',
    'Review Content':'Review_Content',
    'Review date': 'Review_Date',
    'URL': 'URL',
    'Seeding or not': 'Seeding_Flag',
    'Review name': 'Review_Name',
    'People_find_helpful': 'People_Find_Helpful',
    'Syndicated source': 'Syndicated_Source',
    'Comp Model': 'Comp_Model',
    'HP Class': 'HP_Class',
    'Review Model': 'Review_Model',
    'Competitor_Flag': 'Competitor_Flag'
}

# Rename columns
walmart_hp_combine = walmart_hp_combine.rename(columns=column_mapping)
walmart_hp_combine 

# walmart_hp_combine.to_excel('walmart_hp_combine.xlsx')

In [ ]:
Final_review = pd.concat([review_template, walmart_hp_combine], ignore_index = True)
len(Final_review.columns)

Final_review['Review_Date'] = pd.to_datetime(Final_review['Review_Date']).dt.date
Final_review['Response_Date'] = pd.to_datetime(Final_review['Response_Date']).dt.date
Final_review['Review_Rating'] = Final_review['Review_Rating'].astype('int64')
Final_review['People_Find_Helpful'] = Final_review['People_Find_Helpful'].fillna(0).astype('int64')
Final_review['Scraping_Date'] =  pd.to_datetime(Final_review['Scraping_Date']).dt.date

Final_review['Syndicated_Source'].fillna('', inplace=True)

Final_review.info()


In [ ]:
string_columns = Final_review.select_dtypes(include = 'object').columns
Final_review[string_columns] = Final_review[string_columns].fillna('')

## Combine with previous and save to SQL

In [ ]:
# Query previous review
from sqlalchemy import create_engine

server = 'DB-Cluster3.ijp.sgp.rd.hpicorp.net'
database = 'STAR_Rating'
schema = 'dbo'
driver = 'ODBC Driver 17 for SQL Server'

# dataframe = amazon_final_df
table = "Ink_web_reviews"

engine = create_engine(f"mssql+pyodbc://{server}/{database}?driver={driver}", echo=True)


existing_rows_query = f"""
    SELECT *
    FROM {schema}.{table}
    WHERE Retailer in ('Walmart')
"""
result_df = pd.read_sql_query(existing_rows_query, engine)



In [ ]:
result_df['Review_Date'] = pd.to_datetime(result_df['Review_Date']).dt.date

# non_duplicated_df = Final_review[
#                                 (~Final_review['Review_Name'].isin(result_df['Review_Name']))].drop_duplicates()

result_df = result_df[['Review_Model', 'Retailer', 'Review_Name', 'Review_Rating', 'Review_Title', 'Review_Content', 'Review_Date']]

# Merge the DataFrames on both 'Review_Name' and 'Review_Content'
merged_df = pd.merge(Final_review, result_df, on=['Review_Model','Retailer','Review_Name', 'Review_Rating','Review_Title','Review_Content','Review_Date'], how='left', indicator=True)

# Filter the rows where the indicator column is 'left_only'
non_duplicated_df = merged_df[merged_df['_merge'] == 'left_only']

# Drop the indicator column
non_duplicated_df = non_duplicated_df.drop('_merge', axis=1)

# Drop duplicates based on 'Review_Name' and 'Review_Content'
non_duplicated_df = non_duplicated_df.drop_duplicates(subset=['Review_Model','Retailer','Review_Name', 'Review_Rating','Review_Title','Review_Content','Review_Date'])


non_duplicated_df = non_duplicated_df.reset_index(drop=True)


non_duplicated_df.sort_values(by = ['Review_Model','Review_Content'])


In [ ]:
from sqlalchemy import create_engine, text
 
server = 'DB-Cluster3.ijp.sgp.rd.hpicorp.net'
database = 'STAR_Rating'
schema = 'dbo'
driver = 'ODBC Driver 17 for SQL Server'

dataframe = non_duplicated_df
table = "Ink_web_reviews"

engine = create_engine(f"mssql+pyodbc://{server}/{database}?driver={driver}", echo=True)
 
chunk_size = 10000
total_rows = len(dataframe)
num_chunk = (total_rows + chunk_size - 1) // chunk_size

for i in range(num_chunk):
    start_index = i * chunk_size
    end_index = (i + 1) * chunk_size
    chunk = dataframe.iloc[start_index:end_index]
    
    chunk.to_sql(table, engine, index=False, if_exists="append", schema="dbo")
    print(f"Chunk {i+1}/{num_chunk} saved to SQL.")

# HHO

In [ ]:
path = r'C:\Users\TaYu430\anaconda3\envs\webscrap\My Scripts\Star rating scrape URL and info - NPI.xlsx'
sheet = 'HHO'
hho = pd.read_excel(path, sheet_name=sheet)
skus = hho['SKU'].to_list()
len(skus)

In [ ]:
def hho_rating(sku, df = None):
    api_url = 'https://api.bazaarvoice.com/data/products.json'
    params = {
        'resource': 'products',
        'filter': f'id:eq:{sku}',
        # 'filter_reviews': 'contentlocale:eq:en_US,en_CA',
        # 'filter_reviewcomments': 'contentlocale:eq:en_US,en_CA',
        'filteredstats': 'Reviews',
        'stats': 'Reviews,questions,answers',
        'passkey': 'caBZoE5X0dmsywGCMoQmo6OPymWLQFnY37VernuC3SGkY',
        'apiversion': '5.5',
        'displaycode': '8843-en_us'
    }

    response = requests.get(api_url, params=params)

    if response.status_code == 200:
        data = json.loads(response.text)
        
    product_info = data['Results'][0]
    rating_info = product_info['ReviewStatistics']

    if df is None:
        df = pd.DataFrame()
    for rating in rating_info['RatingDistribution']:
            df = df.append(
                {
                 'OriginalProductName': product_info['Name'],
                 'URL': product_info['ProductPageUrl'],
                   'SKU':sku},
             
                ignore_index=True)
            
    return df

In [ ]:
def hho_review(sku):
    
    api_url = 'https://api.bazaarvoice.com/data/reviews.json'

    params = {
        'resource': 'reviews',
        'action': 'REVIEWS_N_STATS',
        'filter': f'productid:eq:{sku}',
        'include': 'authors,products,comments',
        'limit': 100,
        'offset': 0,
        'sort': 'submissiontime:desc',
        'passkey': 'caBZoE5X0dmsywGCMoQmo6OPymWLQFnY37VernuC3SGkY',
        'apiversion': '5.5',
        'displaycode': '8843-en_us'
    }

    response = requests.get(api_url, params=params)

    if response.status_code == 200:
        data = json.loads(response.text)

    limit = 100
    no_batch = math.ceil(data['TotalResults']/limit)
    print('Total review',data['TotalResults'])

    df = pd.DataFrame()
    for x in range(0,no_batch):
        offset = x*limit
#         print('Offset',offset)
        params = {
        'resource': 'reviews',
        'action': 'REVIEWS_N_STATS',
        'filter': f'productid:eq:{sku}',
        'include': 'authors,products,comments',
        'limit': 100,
        'offset': offset,
        'sort': 'submissiontime:desc',
        'passkey': 'caBZoE5X0dmsywGCMoQmo6OPymWLQFnY37VernuC3SGkY',
        'apiversion': '5.5',
        'displaycode': '8843-en_us'
        }

        response = requests.get(api_url, params=params)
        if response.status_code == 200:
            data = json.loads(response.text)
        else:
            print('status code != 200')

        results_data = data['Results']
        df_temp = pd.DataFrame(results_data)
        df = pd.concat([df, df_temp], axis=0)
        time.sleep(3)

    df['SyndicationSource_Name'] = df.apply(lambda row: row['SyndicationSource'].get('Name') if row['IsSyndicated'] else None, axis=1)
    return df



In [ ]:
import warnings
warnings.filterwarnings("ignore")

result_rating = pd.DataFrame()
for sku in skus:
    print(sku)
    result_rating = hho_rating(sku, result_rating)
result_rating.drop_duplicates(inplace = True)

In [ ]:
import math
result_df = pd.DataFrame()
for sku in skus:
    print('Get review', sku)
    data = hho_review(sku)
    print('Review count',len(data))
    if data is not None:
        result_df = pd.concat([result_df,data],axis = 0)
        print('All Review count',len(result_df))
result_df.drop_duplicates(inplace = True)

In [ ]:
result = result_df.copy()
result = result[result['ContentLocale'].isin(['en_US','en_CA'])]

result['Verified_Purchase_Flag'] = result['BadgesOrder'].apply(lambda x: 'Verified Purchaser' if
    isinstance(x, list) and len(x) > 0 and 'verifiedPurchaser' in x else " ")

result['Seeding_Flag'] = result['BadgesOrder'].apply(lambda x: 'Seeding' if
    isinstance(x, list) and len(x) > 0 and 'incentivizedReview' in x else " ")

In [ ]:
# Read data for maching HP model and HP Class
path = r"C:\Users\TaYu430\anaconda3\envs\webscrap\My Scripts\Star rating scrape URL and info - NPI.xlsx"
sheets = 'data_new'
data_new = pd.read_excel(path, sheet_name = sheets)
# data_new['HP Model Number'] =  data_new['HP Model Number'].astype(str)
data_new
# df_amazon.head() 

In [ ]:
result['HP Model Number'] = result['OriginalProductName'].str.extract(r'(\d+e?)')
result_combine = pd.merge(result, data_new, on = "HP Model Number", how = "left" )

from datetime import date  
result_combine['Retailer']="HHO"
result_combine['Scraping_Date'] =pd.to_datetime(date.today())
result_combine['Comp_Model'] = ''
result_combine.rename(columns = {'HP_Model':'Review_Model'},inplace = True)
result_combine


## Save to SQL

In [ ]:
result_combine['Competitor_Flag'] = result_combine['Review_Model'].apply(lambda x: 'No' if 'HP' in x else 'Yes')
result_combine.rename(columns = {'LastModeratedTime':'Review_Date'},inplace = True)
result_combine['Review_Date'] = pd.to_datetime(result_combine['Review_Date']).dt.date

result_combine_url = pd.merge(result_combine, result_rating, on = "SKU", how = "left" )

column_mapping = {
    'UserNickname': 'Review_Name',
      'Rating': 'Review_Rating',
    'Title':'Review_Title',
    'ReviewText':'Review_Content',
    'SyndicationSource_Name':'Syndicated_Source',
    'TotalPositiveFeedbackCount': 'People_Find_Helpful',
    'Syndicated source': 'Syndicated_Source'
}

# Rename columns
result_combine_url = result_combine_url.rename(columns=column_mapping)
result_combine_version = result_combine_url[['Review_Date','Review_Rating', 
       'People_Find_Helpful',  'Review_Content', 'Review_Title', 'Review_Name',
        'Syndicated_Source','Verified_Purchase_Flag', 'Seeding_Flag', 'URL', 
       'Review_Model', 'Comp_Model', 'HP_Class',
       'Segment', 'Retailer', 'Scraping_Date', 'Competitor_Flag']]

# result_combine_version.drop_duplicates(inplace = True)

Final_review = pd.concat([review_template, result_combine_version], ignore_index = True)

Final_review['Country'] = 'US'
Final_review['Review_Date'] = pd.to_datetime(Final_review['Review_Date']).dt.date
Final_review['Review_Rating'] = Final_review['Review_Rating'].astype('int64')
Final_review['People_Find_Helpful'] = Final_review['People_Find_Helpful'].fillna(0).astype('int64')
Final_review['Scraping_Date'] =  pd.to_datetime(Final_review['Scraping_Date']).dt.date

string_columns = Final_review.select_dtypes(include = 'object').columns
Final_review[string_columns] = Final_review[string_columns].fillna('')
Final_review.info()

In [ ]:
from sqlalchemy import create_engine, text
 
server = 'DB-Cluster3.ijp.sgp.rd.hpicorp.net'
database = 'STAR_Rating'
schema = 'dbo'
driver = 'ODBC Driver 17 for SQL Server'

dataframe = Final_review
table = "Ink_web_reviews"

engine = create_engine(f"mssql+pyodbc://{server}/{database}?driver={driver}", echo=True)
 
chunk_size = 10000
total_rows = len(dataframe)
num_chunk = (total_rows + chunk_size - 1) // chunk_size

for i in range(num_chunk):
    start_index = i * chunk_size
    end_index = (i + 1) * chunk_size
    chunk = dataframe.iloc[start_index:end_index]
    
    chunk.to_sql(table, engine, index=False, if_exists="append", schema="dbo")
    print(f"Chunk {i+1}/{num_chunk} saved to SQL.")